Regression model code based upon https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

Majority of the code adapted from [artificial data experiment with case removal.ipynb](https://colab.research.google.com/drive/13eN2qzSAB_wRqzzukBVjqcvt2GayzDCl?authuser=1#scrollTo=pRXm-C2PuCWe) 

In [24]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.134
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [25]:
!cat /proc/meminfo

MemTotal:       13333596 kB
MemFree:         9321620 kB
MemAvailable:   12224724 kB
Buffers:          195244 kB
Cached:          2629064 kB
SwapCached:            0 kB
Active:          1143240 kB
Inactive:        2538784 kB
Active(anon):     643884 kB
Inactive(anon):      372 kB
Active(file):     499356 kB
Inactive(file):  2538412 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               548 kB
Writeback:             0 kB
AnonPages:        857656 kB
Mapped:           407112 kB
Shmem:              1016 kB
Slab:             234580 kB
SReclaimable:     190472 kB
SUnreclaim:        44108 kB
KernelStack:        4672 kB
PageTables:         8380 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666796 kB
Committed_AS:    3804208 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1040 kB
AnonHugePages:   

# Data Preparation

In [26]:
import keras
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle


Code adapted from https://www.kaggle.com/jshih7/car-price-prediction
to read data file and handle nominal attributes

also using a trick from https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas

Main Code

In [27]:
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs


Model code adapted from https://machinelearningmastery.com/cost-sensitive-neural-network-for-imbalanced-classification/

https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

In [28]:
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

def build_DL_model(X_train):
  model = Sequential()
  model.add(Dense(128, input_dim=X_train.shape[1], kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

Adaptation ML-CDH models

Style 2: source problem + problem difference => solution difference

In [29]:
def build_adaptation_model(X_train_source_n_diff):
  model = Sequential()
  model.add(Dense(128, input_dim=len(X_train_source_n_diff[0]), kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

This provides a baseline

In [30]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# estimator = KerasClassifier(build_fn=build_DL_model, epochs=200, batch_size=5, verbose=0)
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, X, y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

The code above is commented out for efficiency

Baseline executation result: 

Baseline: 82.69% (3.34%)

# Testing full test set without removal

In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt


def test_full_without_removal(EAC_adapt,pair_selection, pair_knowledge, verbose_param):
  #pure DL model
  dl_model = build_DL_model()
  # The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  EPOCHS = 1000
  early_history = dl_model.fit(X_train, y_train, 
                      epochs=EPOCHS, validation_data=(X_test, y_test), verbose=0, 
                      callbacks=[early_stop, tfdocs.modeling.EpochDots()])
  
  # X_dl_train, X_dl_valid, y_dl_train, y_dl_valid = train_test_split(X_train_trim, y_train_trim, test_size=0.1, shuffle= True)
  # early_history = dl_model.fit(X_dl_train, y_dl_train, 
  #                     epochs=EPOCHS, validation_data=(X_dl_valid, y_dl_valid), verbose=verbose_param, 
  #                     callbacks=[early_stop, tfdocs.modeling.EpochDots()])


  y_test_pred = dl_model.predict(X_test)
  y_test_pred = tf.one_hot(tf.argmax(y_test_pred, 1), y_test_pred.shape[1])
  dl_loss = accuracy_score(y_test, y_test_pred)

  print("dl last loss: ",early_history.history['val_loss'][-1])

  #pure knn
  knn = KNeighborsClassifier(n_neighbors=3)
  knn.fit(X_train, y_train)
  y_test_pred = knn.predict(X_test)  
  knn_loss = accuracy_score(y_test, y_test_pred)

  #pure CBR retrieval (knn k=1), no need to retrain knn
  X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
  #we only select the best neighbor here!!!
  X_test_retrieved_index = X_test_retrieved_index[:,0]
  X_test_neighbors = X_train.iloc[X_test_retrieved_index]
  y_test_neighbors = y_train.iloc[X_test_retrieved_index]
  retrieval_loss = accuracy_score(y_test, y_test_neighbors)

  if(EAC_adapt==1):
    #EAC retrieval
    X_test_EAC_retrieved_index = knn.kneighbors(X_test, n_neighbors=3)[1]
    # print("X_test_EAC_retrieved_index")
    # print(X_test_EAC_retrieved_index)
    #bug fixing
    # X_test_EAC_neighbors = X_train.iloc[X_test_EAC_retrieved_index]
    # print("X_test_EAC_neighbors")
    # print(X_test_EAC_neighbors)

  #CBR adaptaion ML-CDH learning
  #pool for training pairs
  X_train_for_CDH = X_train
  y_train_for_CDH = y_train


  X_train_pairs_by_class = [[] for _ in range(num_classes)]
  y_train_pairs_by_class = [[] for _ in range(num_classes)]

  def addToPairList(source_y, X_train_pair, target_y):
    for src_class in range(num_classes):
      if source_y[src_class] == 1:
        X_train_pairs_by_class[src_class].append(X_train_pair)
        y_train_pairs_by_class[src_class].append(target_y)
        break

  #Assemble pairs
  if pair_selection == 1:#not updated
    
    knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
    knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
    X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
    #we only select the second best neighbor, the first best neighbor of a case will be itself
    X_train_neighbors_index = X_train_neighbors_index[:,1]
    X_train_neighbors = X_train_for_CDH[X_train_neighbors_index]
    #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
    X_train_differences = X_train_for_CDH - X_train_neighbors
    y_train_neighbors = y_train_for_CDH[X_train_neighbors_index]
    y_train_differences = y_train_for_CDH - y_train_neighbors
    X_train_source_n_diff = np.concatenate((X_train_neighbors, X_train_differences),axis = 1)
  elif pair_selection == 2:#not updated
    X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
    np.random.shuffle(X_train_for_CDH_index) #shuffle in place
    X_train_neighbors = X_train_for_CDH[X_train_for_CDH_index]
    y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
    X_train_differences = X_train_for_CDH - X_train_neighbors
    y_train_differences = y_train_for_CDH - y_train_neighbors
    X_train_source_n_diff = np.concatenate((X_train_neighbors, X_train_differences),axis = 1)
  elif pair_selection == 12 or pair_selection==124:
    #1
    knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
    knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
    X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
    #we only select the second best neighbor, the first best neighbor of a case will be itself
    X_train_neighbors_index = X_train_neighbors_index[:,1]
    X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
    #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
    y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

    for i in range(len(X_train_for_CDH.index)):
      # print(X_train_for_CDH.shape)
      # print(pd.concat([X_train_for_CDH.iloc[[i]], X_train_neighbors1.iloc[[i]]],axis = 1).shape)
      # print(pd.concat([X_train_for_CDH.iloc[[i]], X_train_neighbors1.iloc[[i]]],axis = 1))
      # print(pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],axis = 0).shape)
      # print(pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True))
      # print(pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True).shape)
      #TODO, maybe use series version? 
      # addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[[i]], X_train_neighbors1.iloc[[i]]],axis = 1), y_train_neighbors.iloc[i])
      addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
    #2
    X_train_neighbors2_left = 0
    X_train_neighbors2_right = 0
    y_train_neighbors2_left = 0
    y_train_neighbors2_right = 0
    if pair_selection == 12:#not updated
      X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
      np.random.shuffle(X_train_for_CDH_index) #shuffle in place
      X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
      y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
      X_train_differences2 = X_train_for_CDH - X_train_neighbors2
      y_train_differences2 = y_train_for_CDH - y_train_neighbors
    elif pair_selection ==124:
      left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
      right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
      X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
      y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
      X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
      y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
    for i in range(len(X_train_neighbors2_left.index)):
      addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
    # X_train_pairs1 = np.concatenate((X_train_for_CDH, X_train_neighbors1), axis = 1)
    # X_train_pairs2 = np.concatenate((X_train_neighbors2_left, X_train_neighbors2_right), axis = 1)
    # X_train_pairs = np.concatenate((X_train_pairs1,X_train_pairs2), axis = 0)
    # y_train_pairs = np.concatenate((y_train_neighbors, y_train_neighbors2), axis= 0)
    

  
  #converting pairs data to nparray
  for i in range(num_classes):
    X_train_pairs_by_class[i] = np.asarray(X_train_pairs_by_class[i]) 
    y_train_pairs_by_class[i] = np.asarray(y_train_pairs_by_class[i])
  # print("test1vvfdi:")
  # print(X_train_pairs_by_class[0][0])
  # print("shape", X_train_pairs_by_class[0][0].shape)
  # print(y_train_pairs_by_class[0][0])

  #training adapt models
  adapt_models = [None for _ in range(num_classes)]
  for i in range(num_classes):
    adapt_model = None
    #select training pairs
    X_train_pairs = X_train_pairs_by_class[i]
    y_train_pairs = y_train_pairs_by_class[i]

    # print("Debug 389ngfv")
    # print(len(X_train_pairs))
    # print(len(X_train_pairs[0]))
    # print(len(y_train_pairs))
    # print("length is",len(X_train_pairs[0]))
    adapt_model = build_adaptation_model(X_train_pairs)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
    EPOCHS = 1000
    X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
    early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
                        epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                        callbacks=[early_stop, tfdocs.modeling.EpochDots()])

    print("adapt model last loss: ",early_history.history['val_loss'][-1])
    adapt_models[i] = adapt_model
  #CBR retrieval + adaptation
  # X_test_differences = X_test - X_test_neighbors
  y_adapt_pred = []

  for i in range(len(X_test_neighbors.index)):
    for src_class in range(num_classes):
      if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
        queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
        y_pred = adapt_models[src_class].predict(queryForAdapt)
        y_adapt_pred.append(y_pred[0])
        continue

  y_adapt_pred = np.asarray(y_adapt_pred)
  y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

  retrieval_N_adapt_loss = accuracy_score(y_test, y_adapt_pred)

  #EAC + CDH
  retrieval_N_EAC_adapt_loss = -1
  if(EAC_adapt==1):
    y_adapt_pred = []
    for i in range(len(X_test_EAC_retrieved_index)):
      #for one query
      # voting = [0 for _ in range(num_classes)]
      voting = np.zeros((num_classes,), dtype=int)
      for j in range(len(X_test_EAC_retrieved_index[i])):
        retrieved_index = X_test_EAC_retrieved_index[i][j]
        for src_class in range(num_classes):
          if(y_train.iloc[retrieved_index][src_class] == 1):
            queryForAdapt = np.asarray([pd.concat([X_train.iloc[retrieved_index], X_test.iloc[i]],ignore_index=True)])
            y_pred = adapt_models[src_class].predict(queryForAdapt)
            voting = np.add(voting, y_pred[0])
      y_adapt_pred.append(voting)

    y_adapt_pred = np.asarray(y_adapt_pred)
    y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

    retrieval_N_EAC_adapt_loss = accuracy_score(y_test, y_adapt_pred)
  #normal CDH
  cdh_knns = [KNeighborsClassifier(n_neighbors=1) for _ in range(num_classes)]
  # cdh_knn = KNeighborsClassifier(n_neighbors=1)
  # might need multiple knn here , one for each class.
  for class_index in range(num_classes):
    X_train_pairs = X_train_pairs_by_class[class_index]
    y_train_pairs = y_train_pairs_by_class[class_index]
    cdh_knns[class_index].fit(X_train_pairs, y_train_pairs)
  # y_diff_pred = knn.predict(pd.concat([X_test_neighbors, X_test],axis = 1))  
  y_diff_pred = []
  for i in range(len(X_test_neighbors.index)):
    for src_class in range(num_classes):
      if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
        queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
        y_pred = cdh_knns[src_class].predict(queryForAdapt)
        y_diff_pred.append(y_pred[0])
        continue
  answer_pred_adapted =  y_diff_pred
  answer_pred_adapted = np.asarray(answer_pred_adapted)
  # print("normal Cdh",answer_pred_adapted)
  normal_cdh_loss = accuracy_score(y_test, answer_pred_adapted)

  #CBR retrieval + adaptation
  # X_test_differences = X_test - X_test_neighbors
  # y_adapt_pred = adapt_model.predict(X_test_differences)
  # y_test_pred = y_test_neighbors + y_adapt_pred
  print("")
  print("dl_loss", dl_loss)
  print("knn_loss", knn_loss)
  print("retrieval_loss", retrieval_loss)
  print("retrieval_N_adapt_loss", retrieval_N_adapt_loss)
  print("CBR normal CDH loss", normal_cdh_loss)
  if(EAC_adapt==1):
    print("retrieval_N_EAC_adapt_loss loss", retrieval_N_EAC_adapt_loss)
  # return (dl_model, knn, adapt_model)
  return (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss)

#Separate the training and testing



In [32]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import mean_squared_error
# import tensorflow_docs as tfdocs
# import tensorflow_docs.plots
# import tensorflow_docs.modeling
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# #revamped feature config
# #EAC_adapt:0, no EAC; 1 EAC; 2 C2C-EAC
# #pair_selection: 1, neighbor; 4, random pairs; 5, C2C pairs
# #C2C IS ONLY USABLE WHEN EAC_adapt contains 2 and pair_selection contains 5 
# def train_models(X_train,y_train, EAC_adapt,pair_selection, pair_knowledge, verbose_param):
#   #pure DL model
#   dl_model = build_DL_model(X_train)
#   # The patience parameter is the amount of epochs to check for improvement
#   early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#   EPOCHS = 1000
#   X_dl_train, X_dl_valid, y_dl_train, y_dl_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle= True)
#   early_history = dl_model.fit(X_dl_train, y_dl_train, 
#                       epochs=EPOCHS, validation_data=(X_dl_valid, y_dl_valid), verbose=verbose_param, 
#                       callbacks=[early_stop, tfdocs.modeling.EpochDots()])

#   # plotter.plot({'Early Stopping': early_history}, metric = "mae")
#   # plt.ylabel('MAE')
#   print("dl last loss: ",early_history.history['val_loss'][-1])


#   #pure knn
#   knn = KNeighborsClassifier(n_neighbors=3)
#   knn.fit(X_train, y_train)

#   #knn by class
#   X_train_by_class = [[] for _ in range(num_classes)]
#   y_train_by_class = [[] for _ in range(num_classes)]
#   knn_by_class= [0 for _ in range(num_classes) ]
#   if("2" in EAC_adapt and "5" in pair_selection):
#     for i in range(len(y_train.index)):
#       for src_class in range(num_classes):
#         if y_train.iloc[i][src_class] == 1:
#           X_train_by_class[src_class].append(X_train.iloc[i])
#           y_train_by_class[src_class].append(y_train.iloc[i])
#           break
#     for src_class in range(num_classes):
#       classKnn = KNeighborsClassifier(n_neighbors=1)
#       classKnn.fit(X_train_by_class[src_class], y_train_by_class[src_class])
#       knn_by_class[src_class] = classKnn

#   #CBR adaptaion ML-CDH learning
#   #pool for training pairs
#   X_train_for_CDH = X_train
#   y_train_for_CDH = y_train
#   random_pairs_count = 10 * len(X_train_for_CDH)

#   X_train_pairs_by_class = [[] for _ in range(num_classes)]
#   y_train_pairs_by_class = [[] for _ in range(num_classes)]

#   def addToPairList(source_y, X_train_pair, target_y):
#     for src_class in range(num_classes):
#       if source_y[src_class] == 1:
#         X_train_pairs_by_class[src_class].append(X_train_pair)
#         y_train_pairs_by_class[src_class].append(target_y)
#         break
 
#   #Assemble pairs
#   if "1" in pair_selection:
#     knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
#     knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
#     X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
#     #we only select the second best neighbor, the first best neighbor of a case will be itself
#     X_train_neighbors_index = X_train_neighbors_index[:,1]
#     X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
#     #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
#     y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

#     for i in range(len(X_train_for_CDH.index)):
#       addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
#   if "2" in pair_selection:#not updated
#     #2
#     X_train_neighbors2_left = 0
#     X_train_neighbors2_right = 0
#     y_train_neighbors2_left = 0
#     y_train_neighbors2_right = 0
#     if pair_selection == 12:#not updated
#       X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
#       np.random.shuffle(X_train_for_CDH_index) #shuffle in place
#       X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
#       y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
#       X_train_differences2 = X_train_for_CDH - X_train_neighbors2
#       y_train_differences2 = y_train_for_CDH - y_train_neighbors
#   if "4" in pair_selection:
#     left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
#     right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
#     X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
#     y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
#     X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
#     y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
#     for i in range(len(X_train_neighbors2_left.index)):
#       addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
#   if "5" in pair_selection:#select C2C neighbors
#     for src_class in range(num_classes):#src class
#       for i in range(len(y_train_by_class[src_class])):
#         X_train_neighbors5_left = (np.asarray(X_train_by_class[src_class][i])).reshape(1,-1)
#         y_train_neighbors5_left = y_train_by_class[src_class][i]
#         # y_train_neighbors5_left = np.asarray(y_train_by_class[src_class][i]).reshape(1, -1)

#         for tgt_class in range(num_classes): #target classes
#           if(tgt_class == src_class):#skip
#             continue
#           #.reshape(1, -1)
#           X_test_retrieved_index = knn_by_class[tgt_class].kneighbors(X_train_neighbors5_left)[1]
#           X_test_retrieved_index = X_test_retrieved_index[:,0][0]
#           X_train_neighbors5_right = X_train_by_class[tgt_class][X_test_retrieved_index]
#           y_train_neighbors5_right = y_train_by_class[tgt_class][X_test_retrieved_index]
#           # print(X_train_neighbors5_left)
#           X_train_neighbors5_left_series = pd.Series(X_train_neighbors5_left[0])
#           # print(X_train_neighbors5_left)
#           # print(X_train_neighbors5_right)
          
#           addToPairList(y_train_neighbors5_left, 
#                         pd.concat([X_train_neighbors5_left_series, X_train_neighbors5_right],ignore_index=True),y_train_neighbors5_right)

  
#   #converting pairs data to nparray
#   for i in range(num_classes):
#     X_train_pairs_by_class[i] = np.asarray(X_train_pairs_by_class[i]) 
#     y_train_pairs_by_class[i] = np.asarray(y_train_pairs_by_class[i])
#   # print("test1vvfdi:")
#   # print(X_train_pairs_by_class[0][0])
#   # print("shape", X_train_pairs_by_class[0][0].shape)
#   # print(y_train_pairs_by_class[0][0])

#   #training adapt models
#   adapt_models = [None for _ in range(num_classes)]
#   for i in range(num_classes):
#     adapt_model = None
#     #select training pairs
#     X_train_pairs = X_train_pairs_by_class[i]
#     y_train_pairs = y_train_pairs_by_class[i]

#     # print("Debug 389ngfv")
#     # print(len(X_train_pairs))
#     # print(len(X_train_pairs[0]))
#     # print(len(y_train_pairs))
#     # print("length is",len(X_train_pairs[0]))
#     adapt_model = build_adaptation_model(X_train_pairs)
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
#     EPOCHS = 1000
#     X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
#     early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
#                         epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])

#     print("adapt model last loss: ",early_history.history['val_loss'][-1])
#     adapt_models[i] = adapt_model

#   #normal CDH
#   cdh_knns = [KNeighborsClassifier(n_neighbors=1) for _ in range(num_classes)]
#   # cdh_knn = KNeighborsClassifier(n_neighbors=1)
#   # might need multiple knn here , one for each class.
#   for class_index in range(num_classes):
#     X_train_pairs = X_train_pairs_by_class[class_index]
#     y_train_pairs = y_train_pairs_by_class[class_index]
#     cdh_knns[class_index].fit(X_train_pairs, y_train_pairs)

    
#   return (dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)

testing part


In [33]:
# def test_models(X_train,y_train,X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class,X_train_by_class, y_train_by_class):
#   y_test_pred = dl_model.predict(X_test)
#   y_test_pred = tf.one_hot(tf.argmax(y_test_pred, 1), y_test_pred.shape[1])
#   dl_loss = accuracy_score(y_test, y_test_pred)
#   bal_dl_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_test_pred, 1))

#   y_test_pred = knn.predict(X_test)  
#   knn_loss = accuracy_score(y_test, y_test_pred)
#   bal_knn_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), np.argmax(y_test_pred,axis=1))

#   #pure CBR retrieval (knn k=1), no need to retrain knn
#   X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
#   #we only select the best neighbor here!!!
#   X_test_retrieved_index = X_test_retrieved_index[:,0]
  
#   # print("debugging")
#   # print(len(X_train))
#   # print(max(X_test_retrieved_index))
#   X_test_neighbors = X_train.iloc[X_test_retrieved_index]
#   y_test_neighbors = y_train.iloc[X_test_retrieved_index]
#   retrieval_loss = accuracy_score(y_test, y_test_neighbors)
#   bal_retrieval_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), y_test_neighbors.values.argmax(axis=1))

#   #NN-CDH
#   y_adapt_pred = []
#   for i in range(len(X_test_neighbors.index)):
#     for src_class in range(num_classes):
#       if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
#         queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
#         y_pred = adapt_models[src_class].predict(queryForAdapt)
#         y_adapt_pred.append(y_pred[0])
#         continue
#   y_adapt_pred = np.asarray(y_adapt_pred)
#   y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])
#   retrieval_N_adapt_loss = accuracy_score(y_test, y_adapt_pred)
#   bal_retrieval_N_adapt_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
  
#   #EAC + NN-CDH
#   retrieval_N_EAC_adapt_loss = -1
#   if("1" in EAC_adapt):
#     #EAC retrieval
#     y_adapt_pred = []
#     X_test_EAC_retrieved_index = knn.kneighbors(X_test, n_neighbors=3)[1]
#     for i in range(len(X_test_EAC_retrieved_index)):
#       #for one query
#       # voting = [0 for _ in range(num_classes)]
#       voting = np.zeros((num_classes,), dtype=int)
#       for j in range(len(X_test_EAC_retrieved_index[i])):
#         retrieved_index = X_test_EAC_retrieved_index[i][j]
#         for src_class in range(num_classes):
#           if(y_train.iloc[retrieved_index][src_class] == 1):
#             queryForAdapt = np.asarray([pd.concat([X_train.iloc[retrieved_index], X_test.iloc[i]],ignore_index=True)])
#             y_pred = adapt_models[src_class].predict(queryForAdapt)
#             voting = np.add(voting, y_pred[0])
#       y_adapt_pred.append(voting)

#     y_adapt_pred = np.asarray(y_adapt_pred)
#     y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

#     retrieval_N_EAC_adapt_loss = accuracy_score(y_test, y_adapt_pred)
#     bal_retrieval_N_EAC_adapt_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
#   #C2C-EAC + NN-CDH
#   C2C_EAC_NN_CDH_loss = -1
#   bal_C2C_EAC_NN_CDH_loss = -1
#   if("2" in EAC_adapt):
#     y_adapt_pred = []
#     #retrieval part
#     X_test_EAC_retrieved_index_es = [[] for _ in range(num_classes)]
#     for src_class in range(num_classes):
#       X_test_EAC_retrieved_index_es[src_class] = knn_by_class[src_class].kneighbors(X_test, n_neighbors=1)[1]
#       #we only select the best neighbor here!!!
#       X_test_EAC_retrieved_index_es[src_class]  = X_test_EAC_retrieved_index_es[src_class][:,0]
#     for i in range(len(X_test)):
#       voting = np.zeros((num_classes,), dtype=int)
#       for src_class in range(num_classes):
#         retrieved_index = X_test_EAC_retrieved_index_es[src_class][i]
#         #this index is relative to the class, use X_train_by_class to find it.
#         retrieved_case = X_train_by_class[src_class][retrieved_index]
#         retrieved_sol  = y_train_by_class[src_class][retrieved_index]
#         queryForAdapt = np.asarray([pd.concat([retrieved_case, X_test.iloc[i]],ignore_index=True)])
#         y_pred = adapt_models[src_class].predict(queryForAdapt)
#         voting = np.add(voting, y_pred[0])
#       y_adapt_pred.append(voting)
    
#     y_adapt_pred = np.asarray(y_adapt_pred)
#     y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

#     C2C_EAC_NN_CDH_loss = accuracy_score(y_test, y_adapt_pred)
#     bal_C2C_EAC_NN_CDH_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
#   #TODO

#   #normal CDH
#   y_diff_pred = []
#   for i in range(len(X_test_neighbors.index)):
#     for src_class in range(num_classes):
#       if y_test_neighbors.iloc[i][src_class] == 1:
#         queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
#         y_pred = cdh_knns[src_class].predict(queryForAdapt)
#         y_diff_pred.append(y_pred[0])
#         continue
#   answer_pred_adapted =  y_diff_pred
#   answer_pred_adapted = np.asarray(answer_pred_adapted)
#   # print("normal Cdh",answer_pred_adapted)
#   normal_cdh_loss = accuracy_score(y_test, answer_pred_adapted)
#   bal_normal_cdh_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(answer_pred_adapted, 1))

#   print("")
#   print("dl_loss", dl_loss)
#   print("knn_loss", knn_loss)
#   print("retrieval_loss", retrieval_loss)
#   print("retrieval_N_adapt_loss", retrieval_N_adapt_loss)
#   print("CBR normal CDH loss", normal_cdh_loss)
#   if("1" in EAC_adapt):
#     print("retrieval_N_EAC_adapt_loss loss", retrieval_N_EAC_adapt_loss)
#   if("2" in EAC_adapt):
#     print("C2C_EAC_NN_CDH_loss ", C2C_EAC_NN_CDH_loss)
#   return (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss, C2C_EAC_NN_CDH_loss,
#           bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss, bal_C2C_EAC_NN_CDH_loss)

## Special version for C2C rerun

The following version is for rerun after my mistake with C2C. Do not use in normal circumstances

In [34]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import mean_squared_error
# import tensorflow_docs as tfdocs
# import tensorflow_docs.plots
# import tensorflow_docs.modeling
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# #revamped feature config
# #EAC_adapt:0, no EAC; 1 EAC; 2 C2C-EAC
# #pair_selection: 1, neighbor; 4, random pairs; 5, C2C pairs
# #C2C IS ONLY USABLE WHEN EAC_adapt contains 2 and pair_selection contains 5 
# def train_models(X_train,y_train, EAC_adapt,pair_selection, pair_knowledge, verbose_param):
#   #pure DL model
#   dl_model = None

#   #pure knn
#   knn = KNeighborsClassifier(n_neighbors=3)
#   knn.fit(X_train, y_train)

#   #knn by class
#   X_train_by_class = [[] for _ in range(num_classes)]
#   y_train_by_class = [[] for _ in range(num_classes)]
#   knn_by_class= [0 for _ in range(num_classes) ]
#   if("2" in EAC_adapt and "5" in pair_selection):
#     for i in range(len(y_train.index)):
#       for src_class in range(num_classes):
#         if y_train.iloc[i][src_class] == 1:
#           X_train_by_class[src_class].append(X_train.iloc[i])
#           y_train_by_class[src_class].append(y_train.iloc[i])
#           break
#     for src_class in range(num_classes):
#       classKnn = KNeighborsClassifier(n_neighbors=1)
#       classKnn.fit(X_train_by_class[src_class], y_train_by_class[src_class])
#       knn_by_class[src_class] = classKnn

#   #CBR adaptaion ML-CDH learning
#   #pool for training pairs
#   X_train_for_CDH = X_train
#   y_train_for_CDH = y_train
#   random_pairs_count = 10 * len(X_train_for_CDH)

#   X_train_pairs_by_class = [[] for _ in range(num_classes)]
#   y_train_pairs_by_class = [[] for _ in range(num_classes)]

#   def addToPairList(source_y, X_train_pair, target_y):
#     for src_class in range(num_classes):
#       if source_y[src_class] == 1:
#         X_train_pairs_by_class[src_class].append(X_train_pair)
#         y_train_pairs_by_class[src_class].append(target_y)
#         break
 
#   #Assemble pairs
#   if "1" in pair_selection:
#     knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
#     knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
#     X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
#     #we only select the second best neighbor, the first best neighbor of a case will be itself
#     X_train_neighbors_index = X_train_neighbors_index[:,1]
#     X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
#     #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
#     y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

#     for i in range(len(X_train_for_CDH.index)):
#       addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
#   if "2" in pair_selection:#not updated
#     #2
#     X_train_neighbors2_left = 0
#     X_train_neighbors2_right = 0
#     y_train_neighbors2_left = 0
#     y_train_neighbors2_right = 0
#     if pair_selection == 12:#not updated
#       X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
#       np.random.shuffle(X_train_for_CDH_index) #shuffle in place
#       X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
#       y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
#       X_train_differences2 = X_train_for_CDH - X_train_neighbors2
#       y_train_differences2 = y_train_for_CDH - y_train_neighbors
#   if "4" in pair_selection:
#     left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
#     right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
#     X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
#     y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
#     X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
#     y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
#     for i in range(len(X_train_neighbors2_left.index)):
#       addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
#   if "5" in pair_selection:#select C2C neighbors
#     for src_class in range(num_classes):#src class
#       for i in range(len(y_train_by_class[src_class])):
#         X_train_neighbors5_left = (np.asarray(X_train_by_class[src_class][i])).reshape(1,-1)
#         y_train_neighbors5_left = y_train_by_class[src_class][i]
#         # y_train_neighbors5_left = np.asarray(y_train_by_class[src_class][i]).reshape(1, -1)

#         for tgt_class in range(num_classes): #target classes
#           if(tgt_class == src_class):#skip
#             continue
#           #.reshape(1, -1)
#           X_test_retrieved_index = knn_by_class[tgt_class].kneighbors(X_train_neighbors5_left)[1]
#           X_test_retrieved_index = X_test_retrieved_index[:,0][0]
#           X_train_neighbors5_right = X_train_by_class[tgt_class][X_test_retrieved_index]
#           y_train_neighbors5_right = y_train_by_class[tgt_class][X_test_retrieved_index]
#           # print(X_train_neighbors5_left)
#           X_train_neighbors5_left_series = pd.Series(X_train_neighbors5_left[0])
#           # print(X_train_neighbors5_left)
#           # print(X_train_neighbors5_right)
          
#           addToPairList(y_train_neighbors5_left, 
#                         pd.concat([X_train_neighbors5_left_series, X_train_neighbors5_right],ignore_index=True),y_train_neighbors5_right)

  
#   #converting pairs data to nparray
#   for i in range(num_classes):
#     X_train_pairs_by_class[i] = np.asarray(X_train_pairs_by_class[i]) 
#     y_train_pairs_by_class[i] = np.asarray(y_train_pairs_by_class[i])
#   # print("test1vvfdi:")
#   # print(X_train_pairs_by_class[0][0])
#   # print("shape", X_train_pairs_by_class[0][0].shape)
#   # print(y_train_pairs_by_class[0][0])

#   #training adapt models
#   adapt_models = [None for _ in range(num_classes)]
#   for i in range(num_classes):
#     adapt_model = None
#     #select training pairs
#     X_train_pairs = X_train_pairs_by_class[i]
#     y_train_pairs = y_train_pairs_by_class[i]

#     # print("Debug 389ngfv")
#     # print(len(X_train_pairs))
#     # print(len(X_train_pairs[0]))
#     # print(len(y_train_pairs))
#     # print("length is",len(X_train_pairs[0]))
#     adapt_model = build_adaptation_model(X_train_pairs)
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
#     EPOCHS = 1000
#     X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
#     early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
#                         epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])

#     print("adapt model last loss: ",early_history.history['val_loss'][-1])
#     adapt_models[i] = adapt_model

#   #normal CDH
#   cdh_knns = None

    
#   return (dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)

In [35]:
# def test_models(X_train,y_train,X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class,X_train_by_class, y_train_by_class):

#   dl_loss = 0
#   bal_dl_loss = 0

#   knn_loss = 0
#   bal_knn_loss = 0

#   #pure CBR retrieval (knn k=1), no need to retrain knn
#   # X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
#   # #we only select the best neighbor here!!!
#   # X_test_retrieved_index = X_test_retrieved_index[:,0]
  
#   # # print("debugging")
#   # # print(len(X_train))
#   # # print(max(X_test_retrieved_index))
#   # X_test_neighbors = X_train.iloc[X_test_retrieved_index]
#   # y_test_neighbors = y_train.iloc[X_test_retrieved_index]
#   retrieval_loss = 0
#   bal_retrieval_loss = 0

#   #NN-CDH
#   # y_adapt_pred = []
#   # for i in range(len(X_test_neighbors.index)):
#   #   for src_class in range(num_classes):
#   #     if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
#   #       queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
#   #       y_pred = adapt_models[src_class].predict(queryForAdapt)
#   #       y_adapt_pred.append(y_pred[0])
#   #       continue
#   # y_adapt_pred = np.asarray(y_adapt_pred)
#   # y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])
#   retrieval_N_adapt_loss = 0# accuracy_score(y_test, y_adapt_pred)
#   bal_retrieval_N_adapt_loss =0# balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
  
#   #EAC + NN-CDH
#   retrieval_N_EAC_adapt_loss = -1
#   if("1" in EAC_adapt):
#     # #EAC retrieval
#     # y_adapt_pred = []
#     # X_test_EAC_retrieved_index = knn.kneighbors(X_test, n_neighbors=3)[1]
#     # for i in range(len(X_test_EAC_retrieved_index)):
#     #   #for one query
#     #   # voting = [0 for _ in range(num_classes)]
#     #   voting = np.zeros((num_classes,), dtype=int)
#     #   for j in range(len(X_test_EAC_retrieved_index[i])):
#     #     retrieved_index = X_test_EAC_retrieved_index[i][j]
#     #     for src_class in range(num_classes):
#     #       if(y_train.iloc[retrieved_index][src_class] == 1):
#     #         queryForAdapt = np.asarray([pd.concat([X_train.iloc[retrieved_index], X_test.iloc[i]],ignore_index=True)])
#     #         y_pred = adapt_models[src_class].predict(queryForAdapt)
#     #         voting = np.add(voting, y_pred[0])
#     #   y_adapt_pred.append(voting)

#     # y_adapt_pred = np.asarray(y_adapt_pred)
#     # y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

#     retrieval_N_EAC_adapt_loss = 0 #accuracy_score(y_test, y_adapt_pred)
#     bal_retrieval_N_EAC_adapt_loss = 0# balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
#   #C2C-EAC + NN-CDH
#   C2C_EAC_NN_CDH_loss = -1
#   bal_C2C_EAC_NN_CDH_loss = -1
#   if("2" in EAC_adapt):
#     y_adapt_pred = []
#     #retrieval part
#     X_test_EAC_retrieved_index_es = [[] for _ in range(num_classes)]
#     for src_class in range(num_classes):
#       X_test_EAC_retrieved_index_es[src_class] = knn_by_class[src_class].kneighbors(X_test, n_neighbors=1)[1]
#       #we only select the best neighbor here!!!
#       X_test_EAC_retrieved_index_es[src_class]  = X_test_EAC_retrieved_index_es[src_class][:,0]
#     for i in range(len(X_test)):
#       voting = np.zeros((num_classes,), dtype=int)
#       for src_class in range(num_classes):
#         retrieved_index = X_test_EAC_retrieved_index_es[src_class][i]
#         #this index is relative to the class, use X_train_by_class to find it.
#         retrieved_case = X_train_by_class[src_class][retrieved_index]
#         retrieved_sol  = y_train_by_class[src_class][retrieved_index]
#         queryForAdapt = np.asarray([pd.concat([retrieved_case, X_test.iloc[i]],ignore_index=True)])
#         y_pred = adapt_models[src_class].predict(queryForAdapt)
#         voting = np.add(voting, y_pred[0])
#       y_adapt_pred.append(voting)
    
#     y_adapt_pred = np.asarray(y_adapt_pred)
#     y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

#     C2C_EAC_NN_CDH_loss = accuracy_score(y_test, y_adapt_pred)
#     bal_C2C_EAC_NN_CDH_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
#   #TODO

#   #normal CDH
#   # y_diff_pred = []
#   # for i in range(len(X_test_neighbors.index)):
#   #   for src_class in range(num_classes):
#   #     if y_test_neighbors.iloc[i][src_class] == 1:
#   #       queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
#   #       y_pred = cdh_knns[src_class].predict(queryForAdapt)
#   #       y_diff_pred.append(y_pred[0])
#   #       continue
#   # answer_pred_adapted =  y_diff_pred
#   # answer_pred_adapted = np.asarray(answer_pred_adapted)
#   # # print("normal Cdh",answer_pred_adapted)
#   normal_cdh_loss = 0#accuracy_score(y_test, answer_pred_adapted)
#   bal_normal_cdh_loss =0# balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(answer_pred_adapted, 1))

#   print("")
#   print("dl_loss", dl_loss)
#   print("knn_loss", knn_loss)
#   print("retrieval_loss", retrieval_loss)
#   print("retrieval_N_adapt_loss", retrieval_N_adapt_loss)
#   print("CBR normal CDH loss", normal_cdh_loss)
#   if("1" in EAC_adapt):
#     print("retrieval_N_EAC_adapt_loss loss", retrieval_N_EAC_adapt_loss)
#   if("2" in EAC_adapt):
#     print("C2C_EAC_NN_CDH_loss ", C2C_EAC_NN_CDH_loss)
#   return (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss, C2C_EAC_NN_CDH_loss,
#           bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss, bal_C2C_EAC_NN_CDH_loss)

##Special version for not segmenting training examples

The following is for testing a single adaptation model learning all adaptation examples, not segmented by their source solution. Do not use in normal circumstances.

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#revamped feature config
#EAC_adapt:0, no EAC; 1 EAC; 2 C2C-EAC
#pair_selection: 1, neighbor; 4, random pairs; 5, C2C pairs
#C2C IS ONLY USABLE WHEN EAC_adapt contains 2 and pair_selection contains 5 
def train_models(X_train,y_train, EAC_adapt,pair_selection, pair_knowledge, verbose_param):

  #pure knn
  knn = KNeighborsClassifier(n_neighbors=3)
  knn.fit(X_train, y_train)

  #knn by class
  X_train_by_class = [[] for _ in range(num_classes)]
  y_train_by_class = [[] for _ in range(num_classes)]
  knn_by_class= [0 for _ in range(num_classes) ]
  if("2" in EAC_adapt and "5" in pair_selection):
    for i in range(len(y_train.index)):
      for src_class in range(num_classes):
        if y_train.iloc[i][src_class] == 1:
          X_train_by_class[src_class].append(X_train.iloc[i])
          y_train_by_class[src_class].append(y_train.iloc[i])
          break
    for src_class in range(num_classes):
      classKnn = KNeighborsClassifier(n_neighbors=1)
      classKnn.fit(X_train_by_class[src_class], y_train_by_class[src_class])
      knn_by_class[src_class] = classKnn

  #CBR adaptaion ML-CDH learning
  #pool for training pairs
  X_train_for_CDH = X_train
  y_train_for_CDH = y_train
  random_pairs_count = 10 * len(X_train_for_CDH)

  X_train_pairs = []
  y_train_pairs_source = []
  y_train_pairs_target = []

  def addToPairList(source_y, X_train_pair, target_y):
    X_train_pairs.append(X_train_pair)
    y_train_pairs_source.append(source_y)
    y_train_pairs_target.append(target_y)

  #Assemble pairs
  if "1" in pair_selection:
    knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
    knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
    X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
    #we only select the second best neighbor, the first best neighbor of a case will be itself
    X_train_neighbors_index = X_train_neighbors_index[:,1]
    X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
    #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
    y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

    for i in range(len(X_train_for_CDH.index)):
      addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
  if "2" in pair_selection:#not updated
    #2
    X_train_neighbors2_left = 0
    X_train_neighbors2_right = 0
    y_train_neighbors2_left = 0
    y_train_neighbors2_right = 0
    if pair_selection == 12:#not updated
      X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
      np.random.shuffle(X_train_for_CDH_index) #shuffle in place
      X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
      y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
      X_train_differences2 = X_train_for_CDH - X_train_neighbors2
      y_train_differences2 = y_train_for_CDH - y_train_neighbors
  if "4" in pair_selection:
    left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
    right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
    X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
    y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
    X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
    y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
    for i in range(len(X_train_neighbors2_left.index)):
      addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
  if "5" in pair_selection:#select C2C neighbors
    for src_class in range(num_classes):#src class
      for i in range(len(y_train_by_class[src_class])):
        X_train_neighbors5_left = (np.asarray(X_train_by_class[src_class][i])).reshape(1,-1)
        y_train_neighbors5_left = y_train_by_class[src_class][i]
        # y_train_neighbors5_left = np.asarray(y_train_by_class[src_class][i]).reshape(1, -1)

        for tgt_class in range(num_classes): #target classes
          if(tgt_class == src_class):#skip
            continue
          #.reshape(1, -1)
          X_test_retrieved_index = knn_by_class[tgt_class].kneighbors(X_train_neighbors5_left)[1]
          X_test_retrieved_index = X_test_retrieved_index[:,0][0]
          X_train_neighbors5_right = X_train_by_class[tgt_class][X_test_retrieved_index]
          y_train_neighbors5_right = y_train_by_class[tgt_class][X_test_retrieved_index]
          # print(X_train_neighbors5_left)
          X_train_neighbors5_left_series = pd.Series(X_train_neighbors5_left[0])
          # print(X_train_neighbors5_left)
          # print(X_train_neighbors5_right)
          
          addToPairList(y_train_neighbors5_left, 
                        pd.concat([X_train_neighbors5_left_series, X_train_neighbors5_right],ignore_index=True),y_train_neighbors5_right)

  
  #converting pairs data to nparray
  
  X_train_pairs = np.asarray(X_train_pairs)
  y_train_pairs_source = np.asarray(y_train_pairs_source)
  y_train_pairs_target = np.asarray(y_train_pairs_target)

  # print("test1vvfdi:")
  # print(X_train_pairs_by_class[0][0])
  # print("shape", X_train_pairs_by_class[0][0].shape)
  # print(y_train_pairs_by_class[0][0])

  #training adapt models
  # adapt_models = [None for _ in range(num_classes)]
  adapt_model_without_source = None

  adapt_model_without_source = build_adaptation_model(X_train_pairs)
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
  EPOCHS = 1000
  X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs_target, test_size=0.05, shuffle= True)
  early_history = adapt_model_without_source.fit(X_adapt_train, y_adapt_train, 
                    epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

  print("adapt model (without source) last loss: ",early_history.history['val_loss'][-1])

  # for i in range(num_classes):
  #   adapt_model = None
  #   #select training pairs
  #   X_train_pairs = X_train_pairs_by_class[i]
  #   y_train_pairs = y_train_pairs_by_class[i]

  #   # print("Debug 389ngfv")
  #   # print(len(X_train_pairs))
  #   # print(len(X_train_pairs[0]))
  #   # print(len(y_train_pairs))
  #   # print("length is",len(X_train_pairs[0]))
  #   adapt_model = build_adaptation_model(X_train_pairs)
  #   early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
  #   EPOCHS = 1000
  #   X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
  #   early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
  #                       epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
  #                       callbacks=[early_stop, tfdocs.modeling.EpochDots()])

  #   print("adapt model last loss: ",early_history.history['val_loss'][-1])
  #   adapt_models[i] = adapt_model

  adapt_model_with_source = None

  X_train_pairs = np.concatenate((X_train_pairs, y_train_pairs_source), axis=1)
  adapt_model_with_source = build_adaptation_model(X_train_pairs)
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
  EPOCHS = 1000
  X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs_target, test_size=0.05, shuffle= True)
  early_history = adapt_model_with_source.fit(X_adapt_train, y_adapt_train, 
                    epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

  print("adapt model (with source) last loss: ",early_history.history['val_loss'][-1])

  return (knn, adapt_model_without_source, adapt_model_with_source)

testing part


In [37]:
def test_models(X_train,y_train,X_test,y_test, EAC_adapt, knn, adapt_model_without_source, adapt_model_with_source):

  y_test_pred = knn.predict(X_test)  
  knn_loss = accuracy_score(y_test, y_test_pred)
  bal_knn_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), np.argmax(y_test_pred,axis=1))

  #pure CBR retrieval (knn k=1), no need to retrain knn
  X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
  #we only select the best neighbor here!!!
  X_test_retrieved_index = X_test_retrieved_index[:,0]
  
  # print("debugging")
  # print(len(X_train))
  # print(max(X_test_retrieved_index))
  X_test_neighbors = X_train.iloc[X_test_retrieved_index]
  y_test_neighbors = y_train.iloc[X_test_retrieved_index]
  retrieval_loss = accuracy_score(y_test, y_test_neighbors)
  bal_retrieval_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), y_test_neighbors.values.argmax(axis=1))

  #without source
  y_adapt_pred = []
  for i in range(len(X_test_neighbors.index)):
    queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
    y_pred = adapt_model_without_source.predict(queryForAdapt)
    y_adapt_pred.append(y_pred[0])
    continue
  y_adapt_pred = np.asarray(y_adapt_pred)
  y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])
  without_source_loss = accuracy_score(y_test, y_adapt_pred)
  bal_without_source_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
  #with source
  y_adapt_pred = []
  for i in range(len(X_test_neighbors.index)):
    queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i], y_test_neighbors.iloc[i]],ignore_index=True)])
    y_pred = adapt_model_with_source.predict(queryForAdapt)
    y_adapt_pred.append(y_pred[0])
    continue
  y_adapt_pred = np.asarray(y_adapt_pred)
  y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])
  with_source_loss = accuracy_score(y_test, y_adapt_pred)
  bal_with_source_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  

  print("")
  print("knn_loss", knn_loss)
  print("retrieval_loss", retrieval_loss)
  print("adapt with source", with_source_loss)
  print("adapt without source", without_source_loss)
  return (knn_loss, retrieval_loss, without_source_loss, with_source_loss,
          bal_knn_loss, bal_retrieval_loss, bal_without_source_loss, bal_with_source_loss)

# Graph and report

In [38]:
import seaborn as sns
def plotResults(arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss):
  # kwargs = dict(alpha=0.5, bins=100)
  # plt.hist(arr_dl, **kwargs, color='g', label='dl')
  sns.distplot(arr_dl, hist = False, kde = True, color='g', label='NN',
                  kde_kws = {'linewidth': 3})
  # plt.hist(arr_knn, **kwargs, color='b', label='knn')
  sns.distplot(arr_knn, hist = False, kde = True, color='b', label='k-NN',
                  kde_kws = {'linewidth': 3})
  # plt.hist(arr_retrieval_loss, **kwargs, color='r', label='retrieval')
  sns.distplot(arr_retrieval_loss, hist = False, kde = True, color='r', label='retrieval',
                  kde_kws = {'linewidth': 3})
  # plt.hist(arr_retrieval_N_adapt_loss, **kwargs, color='c', label='retNadapt')
  sns.distplot(arr_retrieval_N_adapt_loss, hist = False, kde = True, color='c', label='CBR + DL-CDH',
                  kde_kws = {'linewidth': 3})
  sns.distplot(arr_normal_cdh_loss, hist = False, kde = True, color='y', label='CBR + normal CDH',
                  kde_kws = {'linewidth': 3})
  plt.gca().set(title='Density plot of loss', xlabel='accuracy')
  plt.xlim(0,1)
  plt.legend()

Maybe distribution plot is biased. Maybe we should show how often CBR beats DL instead.

In [39]:
def reportResults(X_test,arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss):
  adapt_beats_DL = 0
  adapt_beats_DL_beats_ret = 0
  DL_beats_ret = 0
  testedRange = min(too_far, len(X_test))
  for i in range(testedRange):
    if arr_retrieval_N_adapt_loss[i] < arr_dl[i]:
      adapt_beats_DL += 1
      if arr_dl[i] < arr_retrieval_loss [i]:
        adapt_beats_DL_beats_ret += 1
    if arr_dl[i] < arr_retrieval_loss [i]:
      DL_beats_ret += 1
  print("num_cases_removed = ", num_cases_removed)
  print("total: ", testedRange)
  print("adapt_beats_DL: ", adapt_beats_DL)
  print("adapt_beats_DL_beats_ret: ", adapt_beats_DL_beats_ret)
  print("DL_beats_ret: ", DL_beats_ret)

  print("np.mean(arr_dl) = ", np.mean(arr_dl))
  print("np.mean(arr_knn) = ", np.mean(arr_knn))
  print("np.mean(arr_retrieval_loss) = ", np.mean(arr_retrieval_loss))
  print("np.mean(arr_retrieval_N_adapt_loss) = ", np.mean(arr_retrieval_N_adapt_loss))
  print("np.mean(arr_normal_cdh_loss) = ", np.mean(arr_normal_cdh_loss))


In [40]:
def reportResults_kFold(prompt, knn_loss_es, retrieval_loss_es, without_source_loss_es, with_source_loss_es):
  print("reporting k Folds", prompt)
  print("np.mean(knn_loss_es) = ", np.mean(knn_loss_es))
  print("np.mean(retrieval_loss_es) = ", np.mean(retrieval_loss_es))
  print("np.mean(without_source_loss_es) = ", np.mean(without_source_loss_es))
  print("np.mean(with_source_loss_es) = ", np.mean(with_source_loss_es))
  print("")
  print("reporting std", prompt)
  print("np.std(knn_loss_es) = ", np.std(knn_loss_es))
  print("np.std(retrieval_loss_es) = ", np.std(retrieval_loss_es))
  print("np.std(without_source_loss_es) = ", np.std(without_source_loss_es))
  print("np.std(with_source_loss_es) = ", np.std(with_source_loss_es))

In [41]:
def reportGoodBad(X_test,arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss):
  ret = []
  testedRange = min(too_far, len(X_test))
  for i in range(testedRange):
    winner = None
    if arr_dl[i] < arr_knn[i] and arr_dl[i] < arr_retrieval_loss[i] and arr_dl[i] < arr_retrieval_N_adapt_loss[i]:
      winner = "dl"
    elif arr_knn[i] < arr_dl[i] and arr_knn[i] < arr_retrieval_loss[i] and arr_knn[i] < arr_retrieval_N_adapt_loss[i]:
      winner = "knn"
    elif arr_retrieval_loss[i] < arr_dl[i] and arr_retrieval_loss[i] < arr_knn[i] and arr_retrieval_loss[i] < arr_retrieval_N_adapt_loss[i]:
      winner = "Retrieve"
    elif arr_retrieval_N_adapt_loss[i] < arr_dl[i] and arr_retrieval_N_adapt_loss[i] < arr_knn[i] and arr_retrieval_N_adapt_loss[i] < arr_retrieval_loss[i]:
      winner = "Adapt"
    print(X_test[i], winner)
    ret.append((X_test[i], winner))
  return ret

# Result: adapt model 2 + random/close pairs +extra pairs + partial. 


In [ ]:
X_train.shape

(587, 46)

In [ ]:
X_test.shape

(66, 46)

In [ ]:
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = 1
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = 124
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
test_full_without_removal(EAC_adapt,pair_selection, pair_knowledge, 0)


Epoch: 0, accuracy:0.6593,  loss:0.6405,  val_accuracy:0.9091,  val_loss:0.4241,  
..............dl last loss:  0.31456121802330017

Epoch: 0, accuracy:0.6503,  loss:0.6393,  val_accuracy:0.5882,  val_loss:0.7942,  
...................adapt model last loss:  0.4176803231239319

Epoch: 0, accuracy:0.7536,  loss:0.5035,  val_accuracy:0.6316,  val_loss:0.7814,  
.................................adapt model last loss:  0.23807433247566223

dl_loss 0.8939393939393939
knn_loss 0.8484848484848485
retrieval_loss 0.7878787878787878
retrieval_N_adapt_loss 0.8787878787878788
CBR normal CDH loss 0.803030303030303
retrieval_N_EAC_adapt_loss loss 0.8787878787878788


(0.8939393939393939,
 0.8484848484848485,
 0.7878787878787878,
 0.8787878787878788,
 0.803030303030303,
 0.8787878787878788)

#Experiment after separation of training and testing

In [ ]:
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "145"
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
(dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
test_models(X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)


Epoch: 0, accuracy:0.7257,  loss:0.5603,  val_accuracy:0.8333,  val_loss:0.4214,  
....................dl last loss:  0.42026832699775696

Epoch: 0, accuracy:0.5939,  loss:0.7038,  val_accuracy:0.7857,  val_loss:0.5692,  
..........................................adapt model last loss:  0.1687643676996231

Epoch: 0, accuracy:0.5739,  loss:0.6896,  val_accuracy:0.6389,  val_loss:0.5917,  
.................................adapt model last loss:  0.12306912988424301
debugging
587
561

dl_loss 0.8787878787878788
knn_loss 0.8333333333333334
retrieval_loss 0.8484848484848485
retrieval_N_adapt_loss 0.7727272727272727
CBR normal CDH loss 0.7121212121212122
retrieval_N_EAC_adapt_loss loss 0.8636363636363636
C2C_EAC_NN_CDH_loss  0.8636363636363636


(0.8787878787878788,
 0.8333333333333334,
 0.8484848484848485,
 0.7727272727272727,
 0.7121212121212122,
 0.8636363636363636,
 0.8636363636363636,
 0.8820276497695853,
 0.8373271889400922,
 0.8534562211981567,
 0.780184331797235,
 0.7119815668202765,
 0.8658986175115208,
 0.8677419354838709)

# KFOLD CODE

In [42]:
from sklearn.model_selection import KFold
def kFoldExperiment(X,y, num_of_runs):
  # knn_loss, retrieval_loss, without_source_loss, with_source_loss
  knn_loss_es = [] 
  retrieval_loss_es = []
  without_source_loss_es = []
  with_source_loss_es = []

  bal_knn_loss_es = [] 
  bal_retrieval_loss_es = []
  bal_without_source_loss_es = []
  bal_with_source_loss_es = []

  for i in range(num_of_runs):
    kf = KFold(n_splits=10, shuffle=True)
    for train_index, test_index in kf.split(X):
        (X_train,X_test,y_train,y_test) = (None,None,None,None)
        (knn, adapt_model_without_source, adapt_model_with_source) = (None,None,None)
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        (knn, adapt_model_without_source, adapt_model_with_source) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)

        print("weights")
        print(adapt_model_without_source.get_weights())
        
        (knn_loss, retrieval_loss, without_source_loss, with_source_loss,bal_knn_loss, bal_retrieval_loss, bal_without_source_loss, 
         bal_with_source_loss) = test_models(X_train,y_train,X_test,y_test, EAC_adapt, knn, adapt_model_without_source, adapt_model_with_source)
        
        # TODO
        knn_loss_es.append(knn_loss)
        retrieval_loss_es.append(retrieval_loss)
        without_source_loss_es.append(without_source_loss)
        with_source_loss_es.append(with_source_loss)

        bal_knn_loss_es.append(bal_knn_loss)
        bal_retrieval_loss_es.append(bal_retrieval_loss)
        bal_without_source_loss_es.append(bal_without_source_loss)
        bal_with_source_loss_es.append(bal_with_source_loss)
  print("")
  reportResults_kFold("average", knn_loss_es, retrieval_loss_es, without_source_loss_es, with_source_loss_es)
  print("")
  reportResults_kFold("balanced", bal_knn_loss_es, bal_retrieval_loss_es, bal_without_source_loss_es, bal_with_source_loss_es)

In [43]:
# demonstrate data standardization with sklearn
from sklearn.preprocessing import StandardScaler
def scaleX(X, numericCols):
  # create scaler
  Xscaler = StandardScaler()
  scaled_features = X.copy()
  features = scaled_features[numericCols]
  # fit scaler on data
  Xscaler.fit(features.values)
  # apply transform
  features = Xscaler.transform(features.values)
  scaled_features[numericCols] = features
  return scaled_features

# X = scaleX(X,numericCols)

# Dataset loading: credit

In [44]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/crx.data'
nominalCols = [0,3,4,5,6,8,9,11,12] 
numericCols = [1,2,7,10,13,14]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,));

    y = pd.read_csv(filename,header=None, usecols=(15,));

    #remove rows with ? values
    rows_with_na = (X.values == '?').any(1)
    X = X[~rows_with_na]
    y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())

    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Index: []
1       0
2       0
7       0
10      0
13      0
14      0
0_a     0
0_b     0
3_l     0
3_u     0
3_y     0
4_g     0
4_gg    0
4_p     0
5_aa    0
5_c     0
5_cc    0
5_d     0
5_e     0
5_ff    0
5_i     0
5_j     0
5_k     0
5_m     0
5_q     0
5_r     0
5_w     0
5_x     0
6_bb    0
6_dd    0
6_ff    0
6_h     0
6_j     0
6_n     0
6_o     0
6_v     0
6_z     0
8_f     0
8_t     0
9_f     0
9_t     0
11_f    0
11_t    0
12_g    0
12_p    0
12_s    0
dtype: int64


In [ ]:
X.head(5)

,1,2,7,10,13,14,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_f,8_t,9_f,9_t,11_f,11_t,12_g,12_p,12_s
378,0.246524,-0.695675,-0.629144,-0.504019,1.543934,0.664140,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0
582,1.436799,-0.115371,-0.629144,-0.504019,0.265450,-0.193125,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0
339,-0.296200,-0.364215,-0.443604,-0.504019,0.711433,-0.180361,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,1,0,0
228,-1.000390,-0.886787,-0.072523,-0.101174,-0.596784,-0.193125,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
597,-0.845688,0.233011,0.075910,0.100249,-0.596784,-0.018243,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0


In [ ]:
X.isna().sum()
nullseries =  X.isna().sum()
print(nullseries[nullseries > 0])

Series([], dtype: int64)


In [ ]:
X.shape

(653, 46)

In [ ]:
y.shape

(653, 2)

In [ ]:
y.shape[0]

653

In [ ]:
X[1:10]

,1,2,7,10,13,14,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_f,8_t,9_f,9_t,11_f,11_t,12_g,12_p,12_s
421,20.42,1.085,1.500,0,00108,7,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0
119,20.75,10.335,0.335,1,00080,50,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0
347,26.75,4.500,2.500,0,00200,1210,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0
85,37.17,4.000,5.000,0,00280,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1
396,29.83,2.040,0.040,0,00128,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0
368,23.58,11.500,3.000,0,00020,16,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0
463,36.58,0.290,0.000,10,00200,18,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0
109,19.75,0.750,0.795,5,00140,5,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
401,28.92,0.375,0.290,0,00220,140,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0


In [ ]:
y[1:10]

,15_+,15_-
582,1,0
339,0,1
228,1,0
597,1,0
267,0,1
480,0,1
14,1,0
667,0,1
544,0,1


maybe we can preprocess numerical values: normalization and standardization

Based on https://machinelearningmastery.com/how-to-improve-neural-network-stability-and-modeling-performance-with-data-scaling/

According to: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
They used StandardScaler.

IMPORTANT NOTE: Scaling is done on whole dataset instead of on train only. This might be a data leakage issue. However I intentionally do it for now because there is several outlier. If the outliers unluckily all fall in train or testing set, it is going to cause a problem, so I scale them together. 


In [ ]:
X.head(10)

,1,2,7,10,13,14,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_f,8_t,9_f,9_t,11_f,11_t,12_g,12_p,12_s
525,45.17,1.500,2.500,0,00140,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0
421,20.42,1.085,1.500,0,00108,7,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0
119,20.75,10.335,0.335,1,00080,50,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0
347,26.75,4.500,2.500,0,00200,1210,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0
85,37.17,4.000,5.000,0,00280,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1
396,29.83,2.040,0.040,0,00128,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0
368,23.58,11.500,3.000,0,00020,16,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0
463,36.58,0.290,0.000,10,00200,18,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0
109,19.75,0.750,0.795,5,00140,5,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
401,28.92,0.375,0.290,0,00220,140,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0


In [ ]:
y.head()

,15_+,15_-
277,0,1
75,0,1
137,1,0
100,0,1
451,0,1


Split to train and test. random_state involved, so it might produce different results if using default value.

Not using cross validation here yet for simplicity. This is a todo for the future


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state= None)

#Trying 10 fold: Credit dataset

In [ ]:
X.shape

(653, 46)

In [ ]:
X_train.shape

NameError: ignored

In [45]:
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
kFoldExperiment(X,y, 1)


Epoch: 0, accuracy:0.8127,  loss:0.4070,  val_accuracy:0.8470,  val_loss:0.3434,  
.................................................adapt model (without source) last loss:  0.011197518557310104

Epoch: 0, accuracy:0.8183,  loss:0.4076,  val_accuracy:0.8357,  val_loss:0.3524,  
.......................................adapt model (with source) last loss:  0.06514552235603333
weights
[array([[-0.0323457 , -0.1978806 ,  0.14538842, ...,  0.00135315,
         0.0729453 ,  0.13299508],
       [-0.09498184,  0.10818917, -0.33748794, ...,  0.13599534,
         0.19592375,  0.08560415],
       [ 0.07109113,  0.09826579, -0.00847993, ..., -0.12038051,
         0.02418942, -0.08991976],
       ...,
       [ 0.05149938, -0.21317858, -0.09383097, ...,  0.17904241,
         0.12216742, -0.24503495],
       [ 0.07068133, -0.0587214 , -0.1838577 , ..., -0.17582016,
         0.36626557,  0.18430108],
       [ 0.25436744,  0.37027535,  0.26542777, ..., -0.393093  ,
        -0.34294367,  0.46151617]], dt



```
reporting k Folds average
np.mean(knn_loss_es) =  0.8328111888111889
np.mean(retrieval_loss_es) =  0.7987925407925408
np.mean(without_source_loss_es) =  0.8205641025641026
np.mean(with_source_loss_es) =  0.8153100233100232

reporting std average
np.std(knn_loss_es) =  0.044620513012914094
np.std(retrieval_loss_es) =  0.04842702509473827
np.std(without_source_loss_es) =  0.04546265637992197
np.std(with_source_loss_es) =  0.04716968397373342

reporting k Folds balanced
np.mean(knn_loss_es) =  0.8311614407040221
np.mean(retrieval_loss_es) =  0.7969830152636375
np.mean(without_source_loss_es) =  0.8194399852074079
np.mean(with_source_loss_es) =  0.8131693358717937

reporting std balanced
np.std(knn_loss_es) =  0.04476559749708018
np.std(retrieval_loss_es) =  0.04890358244619819
np.std(without_source_loss_es) =  0.045772039445233545
np.std(with_source_loss_es) =  0.04844920547462508
```



# Dataset loading: Balance

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/balance-scale.data'
nominalCols = [] 
numericCols = [1,2,3,4]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(1,2,3,4,));

    y = pd.read_csv(filename,header=None, usecols=(0,));

    #remove rows with ? values
    if(X.values == '?'):
      rows_with_na = (X.values == '?').any(1)
      X = X[~rows_with_na]
      y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [1, 2, 3, 4]
Index: []
1    0
2    0
3    0
4    0
dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':


In [ ]:
X.shape

(625, 4)

In [ ]:
X.head(5)

,1,2,3,4
8,-1.414214,-1.414214,-0.707107,0.707107
502,1.414214,-1.414214,-1.414214,0.000000
164,-0.707107,-0.707107,0.000000,1.414214
246,-0.707107,1.414214,1.414214,-0.707107
228,-0.707107,1.414214,-1.414214,0.707107


In [ ]:
y.head(5)

,0_B,0_L,0_R
8,0,0,1
502,0,1,0
164,0,0,1
246,1,0,0
228,0,1,0


In [ ]:
y.shape

(625, 3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state= None)

# Trying 10 fold: Balance

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
kFoldExperiment(X,y,5)
# kf = KFold(n_splits=10, shuffle=True, random_state=42)
# dl_loss_es=[]
# knn_loss_es = [] 
# retrieval_loss_es = []
# retrieval_N_adapt_loss_es = []
# normal_cdh_loss_es = []
# retrieval_N_EAC_adapt_loss_es = []

# bal_dl_loss_es=[]
# bal_knn_loss_es = [] 
# bal_retrieval_loss_es = []
# bal_retrieval_N_adapt_loss_es = []
# bal_normal_cdh_loss_es = []
# bal_retrieval_N_EAC_adapt_loss_es = []

# for train_index, test_index in kf.split(X):
#     # print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     (dl_model, knn, adapt_models, cdh_knns, knn_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
#     (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss,
#      bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss) = test_models(X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class)
    
#     dl_loss_es.append(dl_loss)
#     knn_loss_es.append(knn_loss)
#     retrieval_loss_es.append(retrieval_loss)
#     retrieval_N_adapt_loss_es.append(retrieval_N_adapt_loss)
#     normal_cdh_loss_es.append(normal_cdh_loss)
#     retrieval_N_EAC_adapt_loss_es.append(retrieval_N_EAC_adapt_loss)

#     bal_dl_loss_es.append(bal_dl_loss)
#     bal_knn_loss_es.append(bal_knn_loss)
#     bal_retrieval_loss_es.append(bal_retrieval_loss)
#     bal_retrieval_N_adapt_loss_es.append(bal_retrieval_N_adapt_loss)
#     bal_normal_cdh_loss_es.append(bal_normal_cdh_loss)
#     bal_retrieval_N_EAC_adapt_loss_es.append(bal_retrieval_N_EAC_adapt_loss)
# reportResults_kFold("average", dl_loss_es,knn_loss_es, retrieval_loss_es,retrieval_N_adapt_loss_es, normal_cdh_loss_es, retrieval_N_EAC_adapt_loss_es)
# reportResults_kFold("balanced", bal_dl_loss_es, bal_knn_loss_es, bal_retrieval_loss_es, 
#                     bal_retrieval_N_adapt_loss_es, bal_normal_cdh_loss_es, bal_retrieval_N_EAC_adapt_loss_es)


Epoch: 0, accuracy:0.8461,  loss:0.4228,  val_accuracy:0.9317,  val_loss:0.2519,  
.........................................................................................adapt model (without source) last loss:  0.0005196684505790472

Epoch: 0, accuracy:0.8412,  loss:0.4124,  val_accuracy:0.9126,  val_loss:0.2354,  
........................................................................................adapt model (with source) last loss:  0.00028704176656901836

knn_loss 0.7936507936507936
retrieval_loss 0.746031746031746
adapt with source 0.9841269841269841
adapt without source 0.9841269841269841

Epoch: 0, accuracy:0.8205,  loss:0.4649,  val_accuracy:0.9098,  val_loss:0.2581,  
..................................adapt model (without source) last loss:  0.006259358488023281

Epoch: 0, accuracy:0.8336,  loss:0.4272,  val_accuracy:0.9235,  val_loss:0.2486,  
....................................................................................................
Epoch: 100, accuracy:1.0000



```
reporting k Folds average
np.mean(knn_loss_es) =  0.7830824372759857
np.mean(retrieval_loss_es) =  0.7728161802355351
np.mean(without_source_loss_es) =  0.9792217101894523
np.mean(with_source_loss_es) =  0.9785611879160265

reporting std average
np.std(knn_loss_es) =  0.042992950988940334
np.std(retrieval_loss_es) =  0.044419274859155575
np.std(without_source_loss_es) =  0.01698825346387991
np.std(with_source_loss_es) =  0.01619228731198973

reporting k Folds balanced
np.mean(knn_loss_es) =  0.5670324023216132
np.mean(retrieval_loss_es) =  0.5597474026155743
np.mean(without_source_loss_es) =  0.9650711935963369
np.mean(with_source_loss_es) =  0.965703475734754

reporting std balanced
np.std(knn_loss_es) =  0.028379529991084753
np.std(retrieval_loss_es) =  0.028264891273725726
np.std(without_source_loss_es) =  0.03736830721868779
np.std(with_source_loss_es) =  0.03435014647369677
```



In [ ]:
np.std([1.0, 0.9206349206349206, 0.9841269841269841, 1.0, 0.9841269841269841, 0.967741935483871, 0.967741935483871, 0.9838709677419355, 0.9838709677419355, 1.0]
)

0.022544506902824456


```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = 1
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = 124
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
reporting k Folds average
np.mean(arr_dl) =  0.9729390681003585
np.mean(arr_knn) =  0.7889400921658986
np.mean(arr_retrieval_loss) =  0.7856118791602663
np.mean(arr_retrieval_N_adapt_loss) =  0.8959549411162314
np.mean(arr_normal_cdh_loss) =  0.7888888888888889
np.mean(retrieval_N_EAC_adapt_loss) =  0.9232206861239117

reporting k Folds balanced
np.mean(arr_dl) =  0.9469392463604986
np.mean(arr_knn) =  0.57911306672938
np.mean(arr_retrieval_loss) =  0.5713817031277086
np.mean(arr_retrieval_N_adapt_loss) =  0.774252877344564
np.mean(arr_normal_cdh_loss) =  0.6273414474125498
np.mean(retrieval_N_EAC_adapt_loss) =  0.8021367288833943
```


Using more pairs produces a lot better result

```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "1"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "124"
random_pairs_count = 6000

dl_loss 1.0
knn_loss 0.7580645161290323
retrieval_loss 0.7580645161290323
retrieval_N_adapt_loss 0.9838709677419355
CBR normal CDH loss 0.7258064516129032
retrieval_N_EAC_adapt_loss loss 0.9838709677419355
reporting k Folds average
np.mean(arr_dl) =  0.9776241679467486
np.mean(arr_knn) =  0.7968509984639016
np.mean(arr_retrieval_loss) =  0.7633640552995391
np.mean(arr_retrieval_N_adapt_loss) =  0.9760112647209421
np.mean(arr_normal_cdh_loss) =  0.7808499743983615
np.mean(retrieval_N_EAC_adapt_loss) =  0.9759856630824373
np.mean(C2C_EAC_NN_CDH_loss) =  -1.0
reporting k Folds balanced
np.mean(arr_dl) =  0.9546992387101767
np.mean(arr_knn) =  0.5762784229508773
np.mean(arr_retrieval_loss) =  0.5512892255629526
np.mean(arr_retrieval_N_adapt_loss) =  0.9744648511841227
np.mean(arr_normal_cdh_loss) =  0.6132406730544281
np.mean(retrieval_N_EAC_adapt_loss) =  0.9588398511841227
np.mean(C2C_EAC_NN_CDH_loss) =  -1.0
```





```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.9712237583205325
np.mean(arr_knn) =  0.7870967741935484
np.mean(arr_retrieval_loss) =  0.7790322580645161
np.mean(arr_retrieval_N_adapt_loss) =  0.969636456733231
np.mean(arr_normal_cdh_loss) =  0.6608550947260625
np.mean(retrieval_N_EAC_adapt_loss) =  0.9744239631336405
np.mean(C2C_EAC_NN_CDH_loss) =  0.967741935483871
reporting k Folds balanced
np.mean(arr_dl) =  0.937722595627719
np.mean(arr_knn) =  0.5676574856948584
np.mean(arr_retrieval_loss) =  0.5616761381193102
np.mean(arr_retrieval_N_adapt_loss) =  0.9574465246258415
np.mean(arr_normal_cdh_loss) =  0.5422665812453158
np.mean(retrieval_N_EAC_adapt_loss) =  0.9662637289269167
np.mean(C2C_EAC_NN_CDH_loss) =  0.9765873015873017
```



Using STD with multi runs



```
reporting k Folds average
np.mean(arr_dl) =  0.9737532002048133
np.mean(arr_knn) =  0.7964925755248334
np.mean(arr_retrieval_loss) =  0.7769943676395289
np.mean(arr_retrieval_N_adapt_loss) =  0.9763184843830007
np.mean(arr_normal_cdh_loss) =  0.6561546338965694
np.mean(retrieval_N_EAC_adapt_loss) =  0.9737685611879161
np.mean(C2C_EAC_NN_CDH_loss) =  0.9516129032258065

reporting std average
np.std(arr_dl) =  0.01971400906942525
np.std(arr_knn) =  0.051138111374122844
np.std(arr_retrieval_loss) =  0.05455483683696814
np.std(arr_retrieval_N_adapt_loss) =  0.017609107718912546
np.std(arr_normal_cdh_loss) =  0.05414928499979276
np.std(retrieval_N_EAC_adapt_loss) =  0.01859286877318543
np.std(C2C_EAC_NN_CDH_loss) =  0.0

reporting k Folds balanced
np.mean(arr_dl) =  0.9591936439458675
np.mean(arr_knn) =  0.57784063014861
np.mean(arr_retrieval_loss) =  0.5630270785334525
np.mean(arr_retrieval_N_adapt_loss) =  0.9663559535837376
np.mean(arr_normal_cdh_loss) =  0.5497414496609297
np.mean(retrieval_N_EAC_adapt_loss) =  0.9639199490045327
np.mean(C2C_EAC_NN_CDH_loss) =  0.9043478260869565

reporting std balanced
np.std(arr_dl) =  0.04885578713837359
np.std(arr_knn) =  0.026486130761340414
np.std(arr_retrieval_loss) =  0.02865470426646368
np.std(arr_retrieval_N_adapt_loss) =  0.045669393639572925
np.std(arr_normal_cdh_loss) =  0.08541824772469106
np.std(retrieval_N_EAC_adapt_loss) =  0.046519168993125026
np.std(C2C_EAC_NN_CDH_loss) =  0.0
```



# Dataset: Car

https://archive.ics.uci.edu/ml/datasets/car+evaluation

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/car.data'
nominalCols = [0,1,2,3,4,5] 
numericCols = []
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,));

    y = pd.read_csv(filename,header=None, usecols=(6,));

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    
    # X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5]
Index: []
0_high     0
0_low      0
0_med      0
0_vhigh    0
1_high     0
1_low      0
1_med      0
1_vhigh    0
2_2        0
2_3        0
2_4        0
2_5more    0
3_2        0
3_4        0
3_more     0
4_big      0
4_med      0
4_small    0
5_high     0
5_low      0
5_med      0
dtype: int64


In [ ]:
X.shape

(1728, 21)

# Trying 10 fold: Car

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)

# kf = KFold(n_splits=10, shuffle=True, random_state=42)
# dl_loss_es=[]
# knn_loss_es = [] 
# retrieval_loss_es = []
# retrieval_N_adapt_loss_es = []
# normal_cdh_loss_es = []
# retrieval_N_EAC_adapt_loss_es = []

# bal_dl_loss_es=[]
# bal_knn_loss_es = [] 
# bal_retrieval_loss_es = []
# bal_retrieval_N_adapt_loss_es = []
# bal_normal_cdh_loss_es = []
# bal_retrieval_N_EAC_adapt_loss_es = []

# for train_index, test_index in kf.split(X):
#     # print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     (dl_model, knn, adapt_models, cdh_knns, knn_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
#     (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss,
#      bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss) = test_models(X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class)
    
#     dl_loss_es.append(dl_loss)
#     knn_loss_es.append(knn_loss)
#     retrieval_loss_es.append(retrieval_loss)
#     retrieval_N_adapt_loss_es.append(retrieval_N_adapt_loss)
#     normal_cdh_loss_es.append(normal_cdh_loss)
#     retrieval_N_EAC_adapt_loss_es.append(retrieval_N_EAC_adapt_loss)

#     bal_dl_loss_es.append(bal_dl_loss)
#     bal_knn_loss_es.append(bal_knn_loss)
#     bal_retrieval_loss_es.append(bal_retrieval_loss)
#     bal_retrieval_N_adapt_loss_es.append(bal_retrieval_N_adapt_loss)
#     bal_normal_cdh_loss_es.append(bal_normal_cdh_loss)
#     bal_retrieval_N_EAC_adapt_loss_es.append(bal_retrieval_N_EAC_adapt_loss)
# reportResults_kFold("average", dl_loss_es,knn_loss_es, retrieval_loss_es,retrieval_N_adapt_loss_es, normal_cdh_loss_es, retrieval_N_EAC_adapt_loss_es)
# reportResults_kFold("balanced", bal_dl_loss_es, bal_knn_loss_es, bal_retrieval_loss_es, 
#                     bal_retrieval_N_adapt_loss_es, bal_normal_cdh_loss_es, bal_retrieval_N_EAC_adapt_loss_es)


Epoch: 0, accuracy:0.8909,  loss:0.2900,  val_accuracy:0.9761,  val_loss:0.0853,  
......................................................................adapt model (without source) last loss:  2.881051557324099e-07

Epoch: 0, accuracy:0.8881,  loss:0.2975,  val_accuracy:0.9679,  val_loss:0.1041,  
....................................................................adapt model (with source) last loss:  7.5409548117022496e-06

knn_loss 0.8265895953757225
retrieval_loss 0.7687861271676301
adapt with source 1.0
adapt without source 1.0

Epoch: 0, accuracy:0.8931,  loss:0.2884,  val_accuracy:0.9780,  val_loss:0.0821,  
................................................................adapt model (without source) last loss:  1.5682760931667872e-05

Epoch: 0, accuracy:0.8888,  loss:0.2943,  val_accuracy:0.9624,  val_loss:0.1011,  
........................................................................adapt model (with source) last loss:  2.567428964539431e-06

knn_loss 0.8728323699421965
ret



```
reporting k Folds average
np.mean(knn_loss_es) =  0.8318282027154187
np.mean(retrieval_loss_es) =  0.7793883586503563
np.mean(without_source_loss_es) =  0.9993043419814492
np.mean(with_source_loss_es) =  0.9991907514450866

reporting std average
np.std(knn_loss_es) =  0.0305214856653105
np.std(retrieval_loss_es) =  0.036613558655121475
np.std(without_source_loss_es) =  0.0022102995652189495
np.std(with_source_loss_es) =  0.0028341388837297846

reporting k Folds balanced
np.mean(knn_loss_es) =  0.6046581154816685
np.mean(retrieval_loss_es) =  0.5565904480242856
np.mean(without_source_loss_es) =  0.9996693764596623
np.mean(with_source_loss_es) =  0.9990325238065197

reporting std balanced
np.std(knn_loss_es) =  0.06009652139821458
np.std(retrieval_loss_es) =  0.08399737641352775
np.std(without_source_loss_es) =  0.0011225236162172084
np.std(with_source_loss_es) =  0.005066217379284004
```





```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = 1
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = 124
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  1.0
np.mean(arr_knn) =  0.8281119774163195
np.mean(arr_retrieval_loss) =  0.8003394273423847
np.mean(arr_retrieval_N_adapt_loss) =  0.8790361607742977
np.mean(arr_normal_cdh_loss) =  0.7297284581260921
np.mean(retrieval_N_EAC_adapt_loss) =  0.9074001881973384

reporting k Folds balanced
np.mean(arr_dl) =  1.0
np.mean(arr_knn) =  0.6022642862673279
np.mean(arr_retrieval_loss) =  0.6162098982135521
np.mean(arr_retrieval_N_adapt_loss) =  0.6585996871178228
np.mean(arr_normal_cdh_loss) =  0.4869124047923922
np.mean(retrieval_N_EAC_adapt_loss) =  0.6974682316945179
```


IMPORTANT NOTE:
Interestingly we get bad accuracy scores here. There are 2 ways to improve
1. more pairs for NN-CDH-EAC  because they only have 100 random pairs per net
EAC style pairs also
C2C-EAC style pairs also
C2C-EAC classification process
2. Using the retrieval of the original EAC. Notice that the knn is performing poorly.  This is a lot worse than the special knn in Vahid's paper.



```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
random_pairs_count = 30000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.999421965317919
np.mean(arr_knn) =  0.8275238607339697
np.mean(arr_retrieval_loss) =  0.7812407581664202
np.mean(arr_retrieval_N_adapt_loss) =  0.9959470358919209
np.mean(arr_normal_cdh_loss) =  0.7019794327194515
np.mean(retrieval_N_EAC_adapt_loss) =  0.9965250705740019
np.mean(C2C_EAC_NN_CDH_loss) =  0.9941860465116279
reporting k Folds balanced
np.mean(arr_dl) =  0.9997826086956522
np.mean(arr_knn) =  0.6089479409264086
np.mean(arr_retrieval_loss) =  0.5769741353708231
np.mean(arr_retrieval_N_adapt_loss) =  0.9857708565589001
np.mean(arr_normal_cdh_loss) =  0.5704297387587864
np.mean(retrieval_N_EAC_adapt_loss) =  0.9849960233297986
np.mean(C2C_EAC_NN_CDH_loss) =  0.99375
```



Down scale to 6000 pairs



```
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.9965284312407581
np.mean(arr_knn) =  0.8275238607339697
np.mean(arr_retrieval_loss) =  0.7812407581664202
np.mean(arr_retrieval_N_adapt_loss) =  0.9837814222341711
np.mean(arr_normal_cdh_loss) =  0.6776986154052964
np.mean(retrieval_N_EAC_adapt_loss) =  0.991897432450598
np.mean(C2C_EAC_NN_CDH_loss) =  0.9883720930232558

reporting k Folds balanced
np.mean(arr_dl) =  0.987081248262441
np.mean(arr_knn) =  0.6089479409264086
np.mean(arr_retrieval_loss) =  0.5769741353708231
np.mean(arr_retrieval_N_adapt_loss) =  0.9508979312902591
np.mean(arr_normal_cdh_loss) =  0.5524140549287838
np.mean(retrieval_N_EAC_adapt_loss) =  0.9698078170915923
np.mean(C2C_EAC_NN_CDH_loss) =  0.9917174796747967
```



Using 10 * training number as random pair number

```
reporting k Folds average
np.mean(arr_dl) =  0.9965250705740019
np.mean(arr_knn) =  0.823440650625084
np.mean(arr_retrieval_loss) =  0.7760182820271542
np.mean(arr_retrieval_N_adapt_loss) =  0.9930535018147599
np.mean(arr_normal_cdh_loss) =  0.6753461486758974
np.mean(retrieval_N_EAC_adapt_loss) =  0.9918974324505981
np.mean(C2C_EAC_NN_CDH_loss) =  0.9941860465116279

reporting k Folds balanced
np.mean(arr_dl) =  0.9984232199181632
np.mean(arr_knn) =  0.6108833970029404
np.mean(arr_retrieval_loss) =  0.5309070704038723
np.mean(arr_retrieval_N_adapt_loss) =  0.9715752359502361
np.mean(arr_normal_cdh_loss) =  0.5354995320623634
np.mean(retrieval_N_EAC_adapt_loss) =  0.9572234387059968
np.mean(C2C_EAC_NN_CDH_loss) =  0.9944444444444445
```



with STD



```
reporting k Folds average
np.mean(arr_dl) =  0.9974559752654926
np.mean(arr_knn) =  0.8313624143029977
np.mean(arr_retrieval_loss) =  0.7810202984272078
np.mean(arr_retrieval_N_adapt_loss) =  0.9890065869068421
np.mean(arr_normal_cdh_loss) =  0.6822832369942198
np.mean(retrieval_N_EAC_adapt_loss) =  0.9936355692969485
np.mean(C2C_EAC_NN_CDH_loss) =  0.9941860465116279

reporting std average
np.std(arr_dl) =  0.005191159835236904
np.std(arr_knn) =  0.02526178528859611
np.std(arr_retrieval_loss) =  0.028800644274238243
np.std(arr_retrieval_N_adapt_loss) =  0.008749778786491984
np.std(arr_normal_cdh_loss) =  0.03448260562534618
np.std(retrieval_N_EAC_adapt_loss) =  0.007427678396361223
np.std(C2C_EAC_NN_CDH_loss) =  0.0

reporting k Folds balanced
np.mean(arr_dl) =  0.9974142678849565
np.mean(arr_knn) =  0.6056267519903155
np.mean(arr_retrieval_loss) =  0.5623310611064158
np.mean(arr_retrieval_N_adapt_loss) =  0.9553387342412119
np.mean(arr_normal_cdh_loss) =  0.5360104748568112
np.mean(retrieval_N_EAC_adapt_loss) =  0.9789716923497828
np.mean(C2C_EAC_NN_CDH_loss) =  0.9980314960629921

reporting std balanced
np.std(arr_dl) =  0.006836961784849787
np.std(arr_knn) =  0.06597657617907499
np.std(arr_retrieval_loss) =  0.07878962568591576
np.std(arr_retrieval_N_adapt_loss) =  0.03835590364768343
np.std(arr_normal_cdh_loss) =  0.0784487264211488
np.std(retrieval_N_EAC_adapt_loss) =  0.028302792154520584
np.std(C2C_EAC_NN_CDH_loss) =  0.0
```



# Dataset: Yeast

Starting from here, the datasets are chosen based on this paper

Anup-to-datecomparisonofstate-of-the-artclassificationalgorithms.pdf

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/yeast.data'
nominalCols = [] 
numericCols = [1,2,3,4,5,6,7,8]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(1,2,3,4,5,6,7,8,),delim_whitespace=True);

    y = pd.read_csv(filename,header=None, usecols=(9,),delim_whitespace=True);

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [1, 2, 3, 4, 5, 6, 7, 8]
Index: []
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64


In [ ]:
X.head(10)

,1,2,3,4,5,6,7,8
1059,-0.438033,1.695697,-0.000389,-0.373480,-0.09759,-0.099131,0.694298,-0.058236
433,-0.875181,-0.322342,0.230449,3.566647,-0.09759,-0.099131,0.001983,-0.152172
742,-0.656607,-0.160899,-1.385414,-0.957203,-0.09759,-0.099131,0.175062,0.693258
1258,-0.875181,-0.645229,-0.000389,-0.519411,-0.09759,-0.099131,-1.036491,-0.527919
575,-0.510891,2.664357,0.576705,-0.446445,-0.09759,-0.099131,-0.517254,1.162942
1121,-0.802323,-0.403064,0.115030,0.210242,-0.09759,-0.099131,0.694298,-0.527919
35,1.747712,1.372811,-0.346645,0.575069,-0.09759,-0.099131,-1.382649,-0.527919
1226,-1.385188,0.081266,-1.385414,-0.592376,-0.09759,-0.099131,-0.690333,0.317511
136,-0.802323,0.646317,-0.000389,-0.008654,-0.09759,-0.099131,0.694298,2.947739
1383,-0.510891,-0.725950,-1.500833,-0.592376,-0.09759,-0.099131,0.694298,0.317511


In [ ]:
y.head(5)

,9_CYT,9_ERL,9_EXC,9_ME1,9_ME2,9_ME3,9_MIT,9_NUC,9_POX,9_VAC
1059,1,0,0,0,0,0,0,0,0,0
433,0,0,0,0,0,0,1,0,0,0
742,0,0,0,0,0,1,0,0,0,0
1258,1,0,0,0,0,0,0,0,0,0
575,0,0,0,0,0,0,0,1,0,0


#Trying 10 fold: Yeast

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.4682,  loss:1.4105,  val_accuracy:0.5064,  val_loss:1.2439,  
......................................................................adapt model (without source) last loss:  0.09153098613023758

Epoch: 0, accuracy:0.4835,  loss:1.3821,  val_accuracy:0.5625,  val_loss:1.1500,  
.....................................................................adapt model (with source) last loss:  0.14591634273529053


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.5436241610738255
retrieval_loss 0.5302013422818792
adapt with source 0.48322147651006714
adapt without source 0.4966442953020134

Epoch: 0, accuracy:0.4913,  loss:1.3513,  val_accuracy:0.5603,  val_loss:1.1766,  
................................................................adapt model (without source) last loss:  0.07541321963071823

Epoch: 0, accuracy:0.4846,  loss:1.3770,  val_accuracy:0.5296,  val_loss:1.2037,  
..................................................................................................adapt model (with source) last loss:  0.09454739093780518

knn_loss 0.4899328859060403
retrieval_loss 0.5167785234899329
adapt with source 0.4228187919463087
adapt without source 0.5033557046979866

Epoch: 0, accuracy:0.4890,  loss:1.3702,  val_accuracy:0.5603,  val_loss:1.1458,  
.................................................................................adapt model (without source) last loss:  0.05556813254952431

Epoch: 0, accuracy:0.4904,  loss:1.3708,  v

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.527027027027027
retrieval_loss 0.5337837837837838
adapt with source 0.5540540540540541
adapt without source 0.49324324324324326

Epoch: 0, accuracy:0.4854,  loss:1.3797,  val_accuracy:0.5412,  val_loss:1.1897,  
...............................................................................adapt model (without source) last loss:  0.061394102871418

Epoch: 0, accuracy:0.4798,  loss:1.3921,  val_accuracy:0.5337,  val_loss:1.1945,  
..............................................................................adapt model (with source) last loss:  0.0918298289179802

knn_loss 0.5135135135135135
retrieval_loss 0.5135135135135135
adapt with source 0.4797297297297297
adapt without source 0.5

Epoch: 0, accuracy:0.4814,  loss:1.3763,  val_accuracy:0.5659,  val_loss:1.1436,  
...............................................................adapt model (without source) last loss:  0.0839841291308403

Epoch: 0, accuracy:0.4877,  loss:1.3515,  val_accuracy:0.5397,  val_loss:1.1664,  
...

...


knn_loss 0.44966442953020136
retrieval_loss 0.5369127516778524
adapt with source 0.5838926174496645
adapt without source 0.5503355704697986

Epoch: 0, accuracy:0.4774,  loss:1.3982,  val_accuracy:0.5408,  val_loss:1.1914,  
......................................................adapt model (without source) last loss:  0.066810742020607

Epoch: 0, accuracy:0.4846,  loss:1.3799,  val_accuracy:0.5311,  val_loss:1.1982,  
..............................................................................................adapt model (with source) last loss:  0.06992741674184799

knn_loss 0.5704697986577181
retrieval_loss 0.5838926174496645
adapt with source 0.5637583892617449
adapt without source 0.5637583892617449

Epoch: 0, accuracy:0.4736,  loss:1.3912,  val_accuracy:0.5494,  val_loss:1.1858,  
....................................................................adapt model (without source) last loss:  0.07320612668991089

Epoch: 0, accuracy:0.4707,  loss:1.3877,  val_accuracy:0.5232,  val_loss

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.5472972972972973
retrieval_loss 0.5405405405405406
adapt with source 0.5540540540540541
adapt without source 0.5

Epoch: 0, accuracy:0.4883,  loss:1.3683,  val_accuracy:0.5569,  val_loss:1.1743,  
.......................................................................adapt model (without source) last loss:  0.08516941219568253

Epoch: 0, accuracy:0.4752,  loss:1.3975,  val_accuracy:0.5449,  val_loss:1.1927,  
....................................................................................adapt model (with source) last loss:  0.10571889579296112

knn_loss 0.4864864864864865
retrieval_loss 0.5540540540540541
adapt with source 0.4797297297297297
adapt without source 0.5

Epoch: 0, accuracy:0.4875,  loss:1.3488,  val_accuracy:0.5644,  val_loss:1.1330,  
..................................................................adapt model (without source) last loss:  0.10699906200170517

Epoch: 0, accuracy:0.4866,  loss:1.3675,  val_accuracy:0.5472,  val_loss:1.1809,  
.............

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.43243243243243246
retrieval_loss 0.5202702702702703
adapt with source 0.4594594594594595
adapt without source 0.4797297297297297

Epoch: 0, accuracy:0.4707,  loss:1.4077,  val_accuracy:0.5262,  val_loss:1.2205,  
..........................................................................................adapt model (without source) last loss:  0.40494707226753235

Epoch: 0, accuracy:0.4687,  loss:1.4026,  val_accuracy:0.5277,  val_loss:1.2135,  
..........................................................................................adapt model (with source) last loss:  0.03259292617440224

knn_loss 0.5
retrieval_loss 0.5067567567567568
adapt with source 0.5337837837837838
adapt without source 0.5337837837837838

reporting k Folds average
np.mean(knn_loss_es) =  0.5001487393433702
np.mean(retrieval_loss_es) =  0.5329022310901506
np.mean(without_source_loss_es) =  0.5218293125340105
np.mean(with_source_loss_es) =  0.517662797025213

reporting std average
np.std(knn_loss_es) =



```
reporting k Folds average
np.mean(knn_loss_es) =  0.5001487393433702
np.mean(retrieval_loss_es) =  0.5329022310901506
np.mean(without_source_loss_es) =  0.5218293125340105
np.mean(with_source_loss_es) =  0.517662797025213

reporting std average
np.std(knn_loss_es) =  0.03784396232280964
np.std(retrieval_loss_es) =  0.030248335102016945
np.std(without_source_loss_es) =  0.03446784956764354
np.std(with_source_loss_es) =  0.04024521859587347

reporting k Folds balanced
np.mean(knn_loss_es) =  0.5031629811821006
np.mean(retrieval_loss_es) =  0.503377359433579
np.mean(without_source_loss_es) =  0.473162395636289
np.mean(with_source_loss_es) =  0.4671051937289704

reporting std balanced
np.std(knn_loss_es) =  0.04815145795431105
np.std(retrieval_loss_es) =  0.04738097658943402
np.std(without_source_loss_es) =  0.05939904402482415
np.std(with_source_loss_es) =  0.06868272794752545
```





```
reporting k Folds average
np.mean(arr_dl) =  0.5787729004171958
np.mean(arr_knn) =  0.5019907491384001
np.mean(arr_retrieval_loss) =  0.519485760928714
np.mean(arr_retrieval_N_adapt_loss) =  0.4629920188645021
np.mean(arr_normal_cdh_loss) =  0.4009341556321422
np.mean(retrieval_N_EAC_adapt_loss) =  0.4960003627788863
np.mean(C2C_EAC_NN_CDH_loss) =  0.6081081081081081

reporting k Folds balanced
np.mean(arr_dl) =  0.497038232059322
np.mean(arr_knn) =  0.4899373868630107
np.mean(arr_retrieval_loss) =  0.48990861672406955
np.mean(arr_retrieval_N_adapt_loss) =  0.41667873642559367
np.mean(arr_normal_cdh_loss) =  0.36369741344219764
np.mean(retrieval_N_EAC_adapt_loss) =  0.4607537585037126
np.mean(C2C_EAC_NN_CDH_loss) =  0.4867682562262906
```



multi runs

```
reporting k Folds average
np.mean(arr_dl) =  0.596252494104843
np.mean(arr_knn) =  0.5028369308906221
np.mean(arr_retrieval_loss) =  0.5350562307273716
np.mean(arr_retrieval_N_adapt_loss) =  0.47281516415744596
np.mean(arr_normal_cdh_loss) =  0.41957826954471245
np.mean(retrieval_N_EAC_adapt_loss) =  0.49868764737892257
np.mean(C2C_EAC_NN_CDH_loss) =  0.5833085434427716

reporting std average
np.std(arr_dl) =  0.041162097045133426
np.std(arr_knn) =  0.044034608484149425
np.std(arr_retrieval_loss) =  0.042307874841011034
np.std(arr_retrieval_N_adapt_loss) =  0.03892795817498548
np.std(arr_normal_cdh_loss) =  0.04360502350468035
np.std(retrieval_N_EAC_adapt_loss) =  0.03727221642946003
np.std(C2C_EAC_NN_CDH_loss) =  0.0402225769494696

reporting k Folds balanced
np.mean(arr_dl) =  0.5413748609381609
np.mean(arr_knn) =  0.5076535748654931
np.mean(arr_retrieval_loss) =  0.5126662977566607
np.mean(arr_retrieval_N_adapt_loss) =  0.44937894844298165
np.mean(arr_normal_cdh_loss) =  0.4060282335696581
np.mean(retrieval_N_EAC_adapt_loss) =  0.4831764787878637
np.mean(C2C_EAC_NN_CDH_loss) =  0.5510401752285099

reporting std balanced
np.std(arr_dl) =  0.07455598914503703
np.std(arr_knn) =  0.06258286037550173
np.std(arr_retrieval_loss) =  0.06872053852069726
np.std(arr_retrieval_N_adapt_loss) =  0.06714262135835385
np.std(arr_normal_cdh_loss) =  0.08040804958316043
np.std(retrieval_N_EAC_adapt_loss) =  0.06759348735481192
np.std(C2C_EAC_NN_CDH_loss) =  0.06323182419886726

```



# Dataset: Seeds

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/seeds_dataset.txt'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,),delim_whitespace=True);

    y = pd.read_csv(filename,header=None, usecols=(7,),delim_whitespace=True);

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[7])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64


In [ ]:
X.head(3)

,0,1,2,3,4,5,6
27,-0.726041,-0.682572,-0.619290,-0.528347,-0.803061,-0.797482,-1.099453
134,0.245448,0.253840,0.479421,0.333629,0.396469,0.847881,0.895208
36,0.465927,0.545509,0.101872,0.446750,0.545084,-0.584811,0.242558


In [ ]:
X.shape

(210, 7)

# Trying 10 fold: Seeds

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.7982,  loss:0.4840,  val_accuracy:0.9512,  val_loss:0.1936,  
..............................................adapt model (without source) last loss:  0.011587092652916908

Epoch: 0, accuracy:0.8106,  loss:0.4460,  val_accuracy:0.9268,  val_loss:0.2116,  
...........................................adapt model (with source) last loss:  0.004469115287065506

knn_loss 0.9047619047619048
retrieval_loss 0.9523809523809523
adapt with source 0.9523809523809523
adapt without source 0.9523809523809523

Epoch: 0, accuracy:0.8308,  loss:0.3871,  val_accuracy:0.9268,  val_loss:0.2162,  
...............................................................adapt model (without source) last loss:  0.001122248126193881

Epoch: 0, accuracy:0.8192,  loss:0.4241,  val_accuracy:0.9024,  val_loss:0.2370,  
....................................................................................................
Epoch: 100, accuracy:1.0000,  loss:0.0000,  val_accuracy:1.0000,  val_loss:0.0000,  
...



```
reporting k Folds average
np.mean(knn_loss_es) =  0.9152380952380953
np.mean(retrieval_loss_es) =  0.9352380952380953
np.mean(without_source_loss_es) =  0.9533333333333335
np.mean(with_source_loss_es) =  0.9533333333333333

reporting std average
np.std(knn_loss_es) =  0.05576920370269467
np.std(retrieval_loss_es) =  0.055336529700319805
np.std(without_source_loss_es) =  0.04516535505287382
np.std(with_source_loss_es) =  0.04311036303907153

reporting k Folds balanced
np.mean(knn_loss_es) =  0.9170184445184445
np.mean(retrieval_loss_es) =  0.9391534391534391
np.mean(without_source_loss_es) =  0.9543791208791209
np.mean(with_source_loss_es) =  0.9544554334554334

reporting std balanced
np.std(knn_loss_es) =  0.05750194746924557
np.std(retrieval_loss_es) =  0.05448333538284092
np.std(without_source_loss_es) =  0.045989718353213895
np.std(with_source_loss_es) =  0.044442472696180145
```





```
reporting k Folds average
np.mean(arr_dl) =  0.8952380952380953
np.mean(arr_knn) =  0.880952380952381
np.mean(arr_retrieval_loss) =  0.8904761904761905
np.mean(arr_retrieval_N_adapt_loss) =  0.9
np.mean(arr_normal_cdh_loss) =  0.8857142857142858
np.mean(retrieval_N_EAC_adapt_loss) =  0.9
np.mean(C2C_EAC_NN_CDH_loss) =  0.9523809523809523

reporting k Folds balanced
np.mean(arr_dl) =  0.8982647907647909
np.mean(arr_knn) =  0.8878270803270801
np.mean(arr_retrieval_loss) =  0.89241822991823
np.mean(arr_retrieval_N_adapt_loss) =  0.9047462722462722
np.mean(arr_normal_cdh_loss) =  0.8898785473785473
np.mean(retrieval_N_EAC_adapt_loss) =  0.9039526214526215
np.mean(C2C_EAC_NN_CDH_loss) =  0.9444444444444445
```



Multi runs



```
reporting k Folds average
np.mean(arr_dl) =  0.9495238095238094
np.mean(arr_knn) =  0.9209523809523809
np.mean(arr_retrieval_loss) =  0.9333333333333331
np.mean(arr_retrieval_N_adapt_loss) =  0.9428571428571427
np.mean(arr_normal_cdh_loss) =  0.8638095238095239
np.mean(retrieval_N_EAC_adapt_loss) =  0.9533333333333333
np.mean(C2C_EAC_NN_CDH_loss) =  0.9485714285714285

reporting std average
np.std(arr_dl) =  0.04986148614365114
np.std(arr_knn) =  0.053545496369030846
np.std(arr_retrieval_loss) =  0.04761904761904762
np.std(arr_retrieval_N_adapt_loss) =  0.05387480237611791
np.std(arr_normal_cdh_loss) =  0.0719094921181435
np.std(retrieval_N_EAC_adapt_loss) =  0.049934197062546024
np.std(C2C_EAC_NN_CDH_loss) =  0.05114560602703829

reporting k Folds balanced
np.mean(arr_dl) =  0.9531132756132756
np.mean(arr_knn) =  0.9259126244126245
np.mean(arr_retrieval_loss) =  0.9365227735227737
np.mean(arr_retrieval_N_adapt_loss) =  0.9475553150553152
np.mean(arr_normal_cdh_loss) =  0.8667980722980723
np.mean(retrieval_N_EAC_adapt_loss) =  0.9563701298701298
np.mean(C2C_EAC_NN_CDH_loss) =  0.9527435527435528

reporting std balanced
np.std(arr_dl) =  0.045773947048098146
np.std(arr_knn) =  0.05110608134934581
np.std(arr_retrieval_loss) =  0.048890137004640136
np.std(arr_retrieval_N_adapt_loss) =  0.0483200172401884
np.std(arr_normal_cdh_loss) =  0.0728159002043527
np.std(retrieval_N_EAC_adapt_loss) =  0.04645073407012013
np.std(C2C_EAC_NN_CDH_loss) =  0.04559015734843396
```



# Dataset: Pima


In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/pima-indians-diabetes.csv'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6,7]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,));

    y = pd.read_csv(filename,header=None, usecols=(8,));

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[8])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7]
Index: []
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64


In [ ]:
X.head(4)

,0,1,2,3,4,5,6,7
133,1.233880,-1.154694,0.253036,0.656358,-0.692891,0.800533,-0.044928,0.490030
722,-0.844885,0.879621,-0.057150,0.530902,0.409837,-0.341740,-0.371101,0.745293
393,0.046014,-0.153185,0.149641,-0.535475,0.062521,-1.255559,-0.026807,0.319855
643,0.046014,-0.966911,-3.572597,-1.288212,-0.692891,-0.506735,0.417150,-0.190672


In [ ]:
y.head(4)

,8_0,8_1
133,1,0
722,0,1
393,1,0
643,1,0


# Trying 10 fold: Pima

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.7147,  loss:0.5346,  val_accuracy:0.7494,  val_loss:0.4976,  
......................................................adapt model (without source) last loss:  0.2958773970603943

Epoch: 0, accuracy:0.7235,  loss:0.5421,  val_accuracy:0.7108,  val_loss:0.5432,  
...........................................adapt model (with source) last loss:  0.32708248496055603

knn_loss 0.6753246753246753
retrieval_loss 0.6623376623376623
adapt with source 0.6883116883116883
adapt without source 0.6883116883116883

Epoch: 0, accuracy:0.7174,  loss:0.5479,  val_accuracy:0.7253,  val_loss:0.5185,  
............................................................................adapt model (without source) last loss:  0.14192257821559906

Epoch: 0, accuracy:0.7286,  loss:0.5390,  val_accuracy:0.7373,  val_loss:0.5291,  
......................................adapt model (with source) last loss:  0.32913151383399963

knn_loss 0.7662337662337663
retrieval_loss 0.7402597402597403
adapt with so



```
reporting k Folds average
np.mean(knn_loss_es) =  0.7320915926179085
np.mean(retrieval_loss_es) =  0.7055023923444976
np.mean(without_source_loss_es) =  0.694002050580998
np.mean(with_source_loss_es) =  0.6955741626794257

reporting std average
np.std(knn_loss_es) =  0.04101238891965368
np.std(retrieval_loss_es) =  0.044095769833236556
np.std(without_source_loss_es) =  0.05017062856948604
np.std(with_source_loss_es) =  0.04932233746049432

reporting k Folds balanced
np.mean(knn_loss_es) =  0.695799438665365
np.mean(retrieval_loss_es) =  0.6692398639974324
np.mean(without_source_loss_es) =  0.6673054518745868
np.mean(with_source_loss_es) =  0.6701881500726156

reporting std balanced
np.std(knn_loss_es) =  0.04857703501098704
np.std(retrieval_loss_es) =  0.04908010944173277
np.std(without_source_loss_es) =  0.05354246634228911
np.std(with_source_loss_es) =  0.055620892268099985
```





```
reporting k Folds average
np.mean(arr_dl) =  0.5536397812713602
np.mean(arr_knn) =  0.6849794941900205
np.mean(arr_retrieval_loss) =  0.6706425153793576
np.mean(arr_retrieval_N_adapt_loss) =  0.5987696514012303
np.mean(arr_normal_cdh_loss) =  0.6042207792207792
np.mean(retrieval_N_EAC_adapt_loss) =  0.583116883116883
np.mean(C2C_EAC_NN_CDH_loss) =  0.6842105263157895

reporting k Folds balanced
np.mean(arr_dl) =  0.5916919162258714
np.mean(arr_knn) =  0.6463337392668883
np.mean(arr_retrieval_loss) =  0.6444572815245712
np.mean(arr_retrieval_N_adapt_loss) =  0.5808719461296186
np.mean(arr_normal_cdh_loss) =  0.5773530645523316
np.mean(retrieval_N_EAC_adapt_loss) =  0.5613729885269192
np.mean(C2C_EAC_NN_CDH_loss) =  0.7043478260869566
```



multi runs


```
reporting k Folds average
np.mean(arr_dl) =  0.7616814764183185
np.mean(arr_knn) =  0.7388072453861928
np.mean(arr_retrieval_loss) =  0.7039131920710866
np.mean(arr_retrieval_N_adapt_loss) =  0.687758031442242
np.mean(arr_normal_cdh_loss) =  0.6458612440191388
np.mean(retrieval_N_EAC_adapt_loss) =  0.694788106630212
np.mean(C2C_EAC_NN_CDH_loss) =  0.7216370471633629

reporting std average
np.std(arr_dl) =  0.0453517428900051
np.std(arr_knn) =  0.04288425528615694
np.std(arr_retrieval_loss) =  0.0483295310141565
np.std(arr_retrieval_N_adapt_loss) =  0.04834789886788082
np.std(arr_normal_cdh_loss) =  0.045993851832649944
np.std(retrieval_N_EAC_adapt_loss) =  0.05250339747400515
np.std(C2C_EAC_NN_CDH_loss) =  0.050103082605795816

reporting k Folds balanced
np.mean(arr_dl) =  0.7300267664938329
np.mean(arr_knn) =  0.7030537273011396
np.mean(arr_retrieval_loss) =  0.6656147236655896
np.mean(arr_retrieval_N_adapt_loss) =  0.6565536802548944
np.mean(arr_normal_cdh_loss) =  0.6130391237107407
np.mean(retrieval_N_EAC_adapt_loss) =  0.6660781715675492
np.mean(C2C_EAC_NN_CDH_loss) =  0.6974413636611423

reporting std balanced
np.std(arr_dl) =  0.04704098129329441
np.std(arr_knn) =  0.04668937207540498
np.std(arr_retrieval_loss) =  0.05305052625028058
np.std(arr_retrieval_N_adapt_loss) =  0.055723330990326
np.std(arr_normal_cdh_loss) =  0.05135387729815293
np.std(retrieval_N_EAC_adapt_loss) =  0.05806275201335232
np.std(C2C_EAC_NN_CDH_loss) =  0.05060338898805063
```



# Dataset: Page-blocks

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/page-blocks.data'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6,7,8,9]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,8,9,),delim_whitespace=True);

    y = pd.read_csv(filename,header=None, usecols=(10,),delim_whitespace=True);

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[10])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Index: []
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64


In [ ]:
X.shape

(5473, 10)

# Trying 10 fold: Page-blocks

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,2)


Epoch: 0, accuracy:0.8945,  loss:0.3187,  val_accuracy:0.9272,  val_loss:0.2181,  
................................................adapt model (without source) last loss:  0.05031164735555649

Epoch: 0, accuracy:0.9012,  loss:0.2996,  val_accuracy:0.9475,  val_loss:0.1678,  
.......................................adapt model (with source) last loss:  0.03699607774615288

knn_loss 0.958029197080292
retrieval_loss 0.958029197080292
adapt with source 0.9416058394160584
adapt without source 0.9379562043795621

Epoch: 0, accuracy:0.8999,  loss:0.3043,  val_accuracy:0.9402,  val_loss:0.1929,  
................................................adapt model (without source) last loss:  0.04621150717139244

Epoch: 0, accuracy:0.9074,  loss:0.2769,  val_accuracy:0.9396,  val_loss:0.1713,  
...............................................................adapt model (with source) last loss:  0.03652339056134224

knn_loss 0.9598540145985401
retrieval_loss 0.9635036496350365
adapt with source 0.9489051



```
reporting k Folds average
np.mean(knn_loss_es) =  0.9662897489958497
np.mean(retrieval_loss_es) =  0.9657429709497057
np.mean(without_source_loss_es) =  0.9595314188873617
np.mean(with_source_loss_es) =  0.9566992820827606

reporting std average
np.std(knn_loss_es) =  0.006915719838279902
np.std(retrieval_loss_es) =  0.0074768215454557195
np.std(without_source_loss_es) =  0.009379114775681959
np.std(with_source_loss_es) =  0.008635922698243611

reporting k Folds balanced
np.mean(knn_loss_es) =  0.8226838376003235
np.mean(retrieval_loss_es) =  0.8393836583993203
np.mean(without_source_loss_es) =  0.7914096566971669
np.mean(with_source_loss_es) =  0.7933965308252512

reporting std balanced
np.std(knn_loss_es) =  0.05430783017630601
np.std(retrieval_loss_es) =  0.06599388413844692
np.std(without_source_loss_es) =  0.07485115955068486
np.std(with_source_loss_es) =  0.05765763301777416
```





```
reporting k Folds average
np.mean(arr_dl) =  0.9539538824910927
np.mean(arr_knn) =  0.956696112838442
np.mean(arr_retrieval_loss) =  0.9579738187058808
np.mean(arr_retrieval_N_adapt_loss) =  0.9349567648353995
np.mean(arr_normal_cdh_loss) =  0.9422583701410481
np.mean(retrieval_N_EAC_adapt_loss) =  0.9395231454916665
np.mean(C2C_EAC_NN_CDH_loss) =  0.9414990859232175

reporting k Folds balanced
np.mean(arr_dl) =  0.712117232970063
np.mean(arr_knn) =  0.7138370403732355
np.mean(arr_retrieval_loss) =  0.7633803995289032
np.mean(arr_retrieval_N_adapt_loss) =  0.6307572269922996
np.mean(arr_normal_cdh_loss) =  0.6442186052273662
np.mean(retrieval_N_EAC_adapt_loss) =  0.7034092808757572
np.mean(C2C_EAC_NN_CDH_loss) =  0.8771153846153845
```



2 times 10 fold

```
reporting k Folds average
np.mean(arr_dl) =  0.9685731061263159
np.mean(arr_knn) =  0.9665633048212545
np.mean(arr_retrieval_loss) =  0.9646435767757776
np.mean(arr_retrieval_N_adapt_loss) =  0.9489289622226076
np.mean(arr_normal_cdh_loss) =  0.9473763661111038
np.mean(retrieval_N_EAC_adapt_loss) =  0.9515787840777165
np.mean(C2C_EAC_NN_CDH_loss) =  0.9675647860259678

reporting std average
np.std(arr_dl) =  0.007996656107944169
np.std(arr_knn) =  0.007818635769009496
np.std(arr_retrieval_loss) =  0.008161736488200913
np.std(arr_retrieval_N_adapt_loss) =  0.011805093942928934
np.std(arr_normal_cdh_loss) =  0.009689588811481999
np.std(retrieval_N_EAC_adapt_loss) =  0.010289273047720357
np.std(C2C_EAC_NN_CDH_loss) =  0.007179361150150107

reporting k Folds balanced
np.mean(arr_dl) =  0.8100202967765615
np.mean(arr_knn) =  0.8269490132087848
np.mean(arr_retrieval_loss) =  0.8404361865499871
np.mean(arr_retrieval_N_adapt_loss) =  0.7361241938606605
np.mean(arr_normal_cdh_loss) =  0.7320485782912092
np.mean(retrieval_N_EAC_adapt_loss) =  0.8012843396089165
np.mean(C2C_EAC_NN_CDH_loss) =  0.8565965349436047

reporting std balanced
np.std(arr_dl) =  0.07519104176013881
np.std(arr_knn) =  0.061361255634633245
np.std(arr_retrieval_loss) =  0.05321437240986915
np.std(arr_retrieval_N_adapt_loss) =  0.0985919717981357
np.std(arr_normal_cdh_loss) =  0.10249295752673333
np.std(retrieval_N_EAC_adapt_loss) =  0.072499686661438
np.std(C2C_EAC_NN_CDH_loss) =  0.06065210656000591

```



# Dataset: Contraceptive Method Choice Data Set

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/cmc.data'
nominalCols = [1,2,6,7] 
numericCols = [0,3,4,5,8]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,8,),sep=',');

    y = pd.read_csv(filename,header=None, usecols=(9,),sep=',');

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[9])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []
0      0
3      0
4      0
5      0
8      0
1_1    0
1_2    0
1_3    0
1_4    0
2_1    0
2_2    0
2_3    0
2_4    0
6_1    0
6_2    0
6_3    0
6_4    0
7_1    0
7_2    0
7_3    0
7_4    0
dtype: int64


In [ ]:
X.head(4)

,0,3,4,5,8,1_1,1_2,1_3,1_4,2_1,2_2,2_3,2_4,6_1,6_2,6_3,6_4,7_1,7_2,7_3,7_4
49,-0.794990,-1.383257,0.419021,-1.729702,-0.282687,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1
1075,1.515194,-1.383257,0.419021,0.578134,-0.282687,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0
659,-0.673401,-0.959123,0.419021,-1.729702,-0.282687,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0
316,-0.551813,-0.534990,0.419021,-1.729702,-0.282687,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0


In [ ]:
X.shape

(1473, 21)

In [ ]:
y.head(4)

,9_1,9_2,9_3
49,1,0,0
1075,1,0,0
659,0,0,1
316,1,0,0


# Trying 10 fold: Contraceptive

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.5084,  loss:0.9738,  val_accuracy:0.5093,  val_loss:0.9567,  
.......................................................adapt model (without source) last loss:  0.43175217509269714

Epoch: 0, accuracy:0.5117,  loss:0.9751,  val_accuracy:0.5012,  val_loss:0.9688,  
...............................................adapt model (with source) last loss:  0.47198253870010376

knn_loss 0.40540540540540543
retrieval_loss 0.4594594594594595
adapt with source 0.527027027027027
adapt without source 0.5202702702702703

Epoch: 0, accuracy:0.4991,  loss:0.9830,  val_accuracy:0.4965,  val_loss:0.9824,  
................................................adapt model (without source) last loss:  0.44167575240135193

Epoch: 0, accuracy:0.5129,  loss:0.9697,  val_accuracy:0.5371,  val_loss:0.9494,  
....................................................adapt model (with source) last loss:  0.5776523351669312

knn_loss 0.4391891891891892
retrieval_loss 0.44594594594594594
adapt with source 0.4



```
reporting k Folds average
np.mean(knn_loss_es) =  0.40922687994116563
np.mean(retrieval_loss_es) =  0.4390853097995956
np.mean(without_source_loss_es) =  0.47440246368817796
np.mean(with_source_loss_es) =  0.4778038242323956

reporting std average
np.std(knn_loss_es) =  0.041485142642806054
np.std(retrieval_loss_es) =  0.04019213237825082
np.std(without_source_loss_es) =  0.03706487925061137
np.std(with_source_loss_es) =  0.03528413502726544

reporting k Folds balanced
np.mean(knn_loss_es) =  0.43909295789697667
np.mean(retrieval_loss_es) =  0.4217047652843515
np.mean(without_source_loss_es) =  0.4502074492215723
np.mean(with_source_loss_es) =  0.4547961071894228

reporting std balanced
np.std(knn_loss_es) =  0.044559776740538175
np.std(retrieval_loss_es) =  0.04310613765723291
np.std(without_source_loss_es) =  0.041952139290926026
np.std(with_source_loss_es) =  0.03979575980737271
```





```
reporting k Folds average
np.mean(arr_dl) =  0.5288656002941717
np.mean(arr_knn) =  0.4256894649751793
np.mean(arr_retrieval_loss) =  0.4500873322301894
np.mean(arr_retrieval_N_adapt_loss) =  0.49218606361463496
np.mean(arr_normal_cdh_loss) =  0.39440614083471226
np.mean(retrieval_N_EAC_adapt_loss) =  0.49153337010479864
np.mean(C2C_EAC_NN_CDH_loss) =  0.54421768707483
reporting k Folds balanced
np.mean(arr_dl) =  0.49578508188150217
np.mean(arr_knn) =  0.44855540994875354
np.mean(arr_retrieval_loss) =  0.43454317477251464
np.mean(arr_retrieval_N_adapt_loss) =  0.4699601045679717
np.mean(arr_normal_cdh_loss) =  0.3871058606330141
np.mean(retrieval_N_EAC_adapt_loss) =  0.47140368648915293
np.mean(C2C_EAC_NN_CDH_loss) =  0.5100198412698412
```



Multi run

```
reporting k Folds average
np.mean(arr_dl) =  0.547317521603236
np.mean(arr_knn) =  0.4096240117668689
np.mean(arr_retrieval_loss) =  0.4347416804559661
np.mean(arr_retrieval_N_adapt_loss) =  0.4794392351535209
np.mean(arr_normal_cdh_loss) =  0.3880400808972238
np.mean(retrieval_N_EAC_adapt_loss) =  0.47820922963780105
np.mean(C2C_EAC_NN_CDH_loss) =  0.513914322485751

reporting std average
np.std(arr_dl) =  0.03810123623451787
np.std(arr_knn) =  0.03750680710336985
np.std(arr_retrieval_loss) =  0.03741967681925101
np.std(arr_retrieval_N_adapt_loss) =  0.04098959878103348
np.std(arr_normal_cdh_loss) =  0.03614251947842028
np.std(retrieval_N_EAC_adapt_loss) =  0.04481613497029232
np.std(C2C_EAC_NN_CDH_loss) =  0.043132229423507876

reporting k Folds balanced
np.mean(arr_dl) =  0.5229381141508201
np.mean(arr_knn) =  0.43883572984731123
np.mean(arr_retrieval_loss) =  0.4178267704715454
np.mean(arr_retrieval_N_adapt_loss) =  0.4548793652709406
np.mean(arr_normal_cdh_loss) =  0.3839333954072596
np.mean(retrieval_N_EAC_adapt_loss) =  0.45469448089499553
np.mean(C2C_EAC_NN_CDH_loss) =  0.4969222855097253

reporting std balanced
np.std(arr_dl) =  0.038280534421347635
np.std(arr_knn) =  0.033924939097700425
np.std(arr_retrieval_loss) =  0.03649325651929315
np.std(arr_retrieval_N_adapt_loss) =  0.041709425475082475
np.std(arr_normal_cdh_loss) =  0.03847498751013724
np.std(retrieval_N_EAC_adapt_loss) =  0.04518575093262454
np.std(C2C_EAC_NN_CDH_loss) =  0.04314633544083794
```



# Dataset: Abalone

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/abalone.data'
nominalCols = [0] 
numericCols = [1,2,3,4,5,6,7]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,),sep=',');

    y = pd.read_csv(filename,header=None, usecols=(8,),sep=',');

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[8])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

# Trying 10 fold: Abalone

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y)

# Dataset: Wine-white

In [21]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/winequality-white.csv'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6,7,8,9,10]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=0, names=[0,1,2,3,4,5,6,7,8,9,10,11], usecols=(0,1,2,3,4,5,6,7,8,9,10,),sep=';');

    y = pd.read_csv(filename,header=0, names=[0,1,2,3,4,5,6,7,8,9,10,11], usecols=(11,),sep=';');

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[11])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Index: []
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64


In [ ]:
X.shape

# Trying 10 fold: Wine-white

In [22]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,2)


Epoch: 0, accuracy:0.4970,  loss:1.1949,  val_accuracy:0.5653,  val_loss:0.9986,  
...........................................................................................adapt model (without source) last loss:  0.014275607652962208

Epoch: 0, accuracy:0.5054,  loss:1.1880,  val_accuracy:0.5546,  val_loss:1.0093,  
..........................................................adapt model (with source) last loss:  0.07762566208839417

knn_loss 0.5673469387755102
retrieval_loss 0.6510204081632653
adapt with source 0.6204081632653061
adapt without source 0.6326530612244898

Epoch: 0, accuracy:0.5023,  loss:1.1710,  val_accuracy:0.5767,  val_loss:0.9854,  
.....................................................adapt model (without source) last loss:  0.03174590691924095

Epoch: 0, accuracy:0.4979,  loss:1.1936,  val_accuracy:0.5671,  val_loss:1.0006,  
....................................................................adapt model (with source) last loss:  0.03917575627565384

knn_loss 0.542

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.5480572597137015
retrieval_loss 0.6646216768916156
adapt with source 0.6359918200408998
adapt without source 0.6625766871165644

Epoch: 0, accuracy:0.5081,  loss:1.1749,  val_accuracy:0.5671,  val_loss:1.0003,  
.........................................................adapt model (without source) last loss:  0.02944169007241726

Epoch: 0, accuracy:0.5047,  loss:1.1845,  val_accuracy:0.5671,  val_loss:1.0027,  
............................................................adapt model (with source) last loss:  0.07129369676113129

knn_loss 0.5163265306122449
retrieval_loss 0.6408163265306123
adapt with source 0.5857142857142857
adapt without source 0.6183673469387755

Epoch: 0, accuracy:0.4984,  loss:1.1852,  val_accuracy:0.5743,  val_loss:0.9831,  
.................................................................adapt model (without source) last loss:  0.0754113718867302

Epoch: 0, accuracy:0.4971,  loss:1.1894,  val_accuracy:0.5620,  val_loss:1.0090,  
.......................

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.563265306122449
retrieval_loss 0.6510204081632653
adapt with source 0.6469387755102041
adapt without source 0.6183673469387755

Epoch: 0, accuracy:0.4933,  loss:1.2013,  val_accuracy:0.5751,  val_loss:0.9920,  
.......................................................adapt model (without source) last loss:  0.0357719287276268

Epoch: 0, accuracy:0.4923,  loss:1.1984,  val_accuracy:0.5466,  val_loss:1.0109,  
.....................................................adapt model (with source) last loss:  0.06344885379076004


...


knn_loss 0.5530612244897959
retrieval_loss 0.6612244897959184
adapt with source 0.6571428571428571
adapt without source 0.6591836734693878

Epoch: 0, accuracy:0.4998,  loss:1.1858,  val_accuracy:0.5487,  val_loss:1.0026,  
.........................................................................adapt model (without source) last loss:  0.03058667853474617

Epoch: 0, accuracy:0.4982,  loss:1.1868,  val_accuracy:0.5450,  val_loss:1.0236,  
.......................................................adapt model (with source) last loss:  0.025437848642468452

knn_loss 0.5448979591836735
retrieval_loss 0.6551020408163265
adapt with source 0.6469387755102041
adapt without source 0.6510204081632653

Epoch: 0, accuracy:0.4969,  loss:1.1911,  val_accuracy:0.5567,  val_loss:1.0118,  
..........................................................................adapt model (without source) last loss:  0.030393466353416443

Epoch: 0, accuracy:0.4993,  loss:1.1804,  val_accuracy:0.5527,  val_loss:1.0136,  


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.5346938775510204
retrieval_loss 0.6693877551020408
adapt with source 0.6061224489795919
adapt without source 0.6142857142857143

Epoch: 0, accuracy:0.5024,  loss:1.1794,  val_accuracy:0.5634,  val_loss:0.9828,  
....................................................adapt model (without source) last loss:  0.04554031789302826

Epoch: 0, accuracy:0.4957,  loss:1.2003,  val_accuracy:0.5564,  val_loss:1.0081,  
..................................................................adapt model (with source) last loss:  0.07996558398008347

knn_loss 0.5755102040816327
retrieval_loss 0.6816326530612244
adapt with source 0.6306122448979592
adapt without source 0.6183673469387755

Epoch: 0, accuracy:0.4911,  loss:1.2038,  val_accuracy:0.5542,  val_loss:1.0170,  
.........................................................................adapt model (without source) last loss:  0.038963500410318375

Epoch: 0, accuracy:0.4937,  loss:1.1948,  val_accuracy:0.5566,  val_loss:1.0129,  
............

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



knn_loss 0.5276073619631901
retrieval_loss 0.6441717791411042
adapt with source 0.6625766871165644
adapt without source 0.6319018404907976

Epoch: 0, accuracy:0.5028,  loss:1.1763,  val_accuracy:0.5792,  val_loss:0.9689,  
..................................................................adapt model (without source) last loss:  0.04017262905836105

Epoch: 0, accuracy:0.5036,  loss:1.1761,  val_accuracy:0.5550,  val_loss:0.9997,  
...............................................................adapt model (with source) last loss:  0.10610046982765198

knn_loss 0.5541922290388548
retrieval_loss 0.6503067484662577
adapt with source 0.6359918200408998
adapt without source 0.6503067484662577

reporting k Folds average
np.mean(knn_loss_es) =  0.5411403948082301
np.mean(retrieval_loss_es) =  0.6564947205876217
np.mean(without_source_loss_es) =  0.6353666374525271
np.mean(with_source_loss_es) =  0.6344480614331622

reporting std average
np.std(knn_loss_es) =  0.01661991426408846
np.std(retriev



```
reporting k Folds average
np.mean(knn_loss_es) =  0.5411403948082301
np.mean(retrieval_loss_es) =  0.6564947205876217
np.mean(without_source_loss_es) =  0.6353666374525271
np.mean(with_source_loss_es) =  0.6344480614331622

reporting std average
np.std(knn_loss_es) =  0.01661991426408846
np.std(retrieval_loss_es) =  0.014809487167621104
np.std(without_source_loss_es) =  0.01904143347537029
np.std(with_source_loss_es) =  0.02007246638036271

reporting k Folds balanced
np.mean(knn_loss_es) =  0.3262180696407081
np.mean(retrieval_loss_es) =  0.4397147835716151
np.mean(without_source_loss_es) =  0.4200064497200807
np.mean(with_source_loss_es) =  0.4207293535305513

reporting std balanced
np.std(knn_loss_es) =  0.04398173874075522
np.std(retrieval_loss_es) =  0.03173939894707622
np.std(without_source_loss_es) =  0.033550358122329496
np.std(with_source_loss_es) =  0.0505880233812723
```





```
reporting k Folds average
np.mean(arr_dl) =  0.4665381244522349
np.mean(arr_knn) =  0.4697909102291223
np.mean(arr_retrieval_loss) =  0.5979950753307457
np.mean(arr_retrieval_N_adapt_loss) =  0.31564959726221775
np.mean(arr_normal_cdh_loss) =  0.41343766954634614
np.mean(retrieval_N_EAC_adapt_loss) =  0.3760782104252744
np.mean(C2C_EAC_NN_CDH_loss) =  0.523517382413088

reporting k Folds balanced
np.mean(arr_dl) =  0.23369949465188347
np.mean(arr_knn) =  0.29233955656831256
np.mean(arr_retrieval_loss) =  0.41643922242289994
np.mean(arr_retrieval_N_adapt_loss) =  0.22825821933393803
np.mean(arr_normal_cdh_loss) =  0.26221174928319757
np.mean(retrieval_N_EAC_adapt_loss) =  0.2924976075057458
np.mean(C2C_EAC_NN_CDH_loss) =  0.23889882416820804
```





```
reporting k Folds average
np.mean(arr_dl) =  0.5763580401485748
np.mean(arr_knn) =  0.5425783564959727
np.mean(arr_retrieval_loss) =  0.6569103960602646
np.mean(arr_retrieval_N_adapt_loss) =  0.5504342473185594
np.mean(arr_normal_cdh_loss) =  0.46938107758440795
np.mean(retrieval_N_EAC_adapt_loss) =  0.6035167563958099
np.mean(C2C_EAC_NN_CDH_loss) =  0.6354741037519303

reporting std average
np.std(arr_dl) =  0.02179267686442935
np.std(arr_knn) =  0.02165176555679546
np.std(arr_retrieval_loss) =  0.022611759724934604
np.std(arr_retrieval_N_adapt_loss) =  0.024432698325331216
np.std(arr_normal_cdh_loss) =  0.01985915156665192
np.std(retrieval_N_EAC_adapt_loss) =  0.018426137178453882
np.std(C2C_EAC_NN_CDH_loss) =  0.026971598082991023

reporting k Folds balanced
np.mean(arr_dl) =  0.3205154850474546
np.mean(arr_knn) =  0.3526121897166673
np.mean(arr_retrieval_loss) =  0.45766137146761016
np.mean(arr_retrieval_N_adapt_loss) =  0.3313981687459058
np.mean(arr_normal_cdh_loss) =  0.29467081156585084
np.mean(retrieval_N_EAC_adapt_loss) =  0.43030951394650324
np.mean(C2C_EAC_NN_CDH_loss) =  0.43502942456604626

reporting std balanced
np.std(arr_dl) =  0.03753660412843252
np.std(arr_knn) =  0.07687480684309066
np.std(arr_retrieval_loss) =  0.0630762456599508
np.std(arr_retrieval_N_adapt_loss) =  0.04080097523486786
np.std(arr_normal_cdh_loss) =  0.02631982162006131
np.std(retrieval_N_EAC_adapt_loss) =  0.06001922188242892
np.std(C2C_EAC_NN_CDH_loss) =  0.054356441048081613
```

